In [35]:
import re
import nltk
import spacy
from transformers import pipeline

# Download NLTK resources (stopwords for Spanish)
nltk.download('stopwords')

# Load spaCy model for Spanish
nlp = spacy.load('es_core_news_sm')

def preprocess_text(text, undesired_characters=[], custom_expressions=[], remove_stopwords=True, remove_ner=True, max_chunk_length=512):
    # Lowercasing
    text = text.lower()

    # Remove undesired phrases
    for expression in custom_expressions:
        text = text.replace(expression, '')

    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text using spaCy
    tokens = [token.text for token in nlp(text)]

    # Remove custom list of expressions
    tokens = [word for word in tokens if word not in custom_expressions]

    # Remove stopwords if remove_stopwords is True
    if remove_stopwords:
        stop_words = set(nltk.corpus.stopwords.words('spanish') + undesired_characters)
        tokens = [word for word in tokens if word.lower() not in stop_words]

    # Perform lemmatization using spaCy
    lemmatized_tokens = [token.lemma_ for token in nlp(' '.join(tokens))]

    # Join the lemmatized tokens back into text
    preprocessed_text = ' '.join(lemmatized_tokens)

    # Remove Named Entities if remove_ner is True
    if remove_ner:
        # Define the NER pipeline
        nlp_ner = pipeline(
            "ner",
            model="mrm8488/bert-spanish-cased-finetuned-ner",
            tokenizer=(
                'mrm8488/bert-spanish-cased-finetuned-ner',
                {"use_fast": False}
            )
        )

        # Split the text into chunks
        chunks = [preprocessed_text[i:i + max_chunk_length] for i in range(0, len(preprocessed_text), max_chunk_length)]

        # Process each chunk
        processed_chunks = []

        for chunk in chunks:
            entities = nlp_ner(chunk)
            entity_positions = set((entity['start'], entity['end']) for entity in entities)

            # Split the chunk into smaller chunks while preserving entities
            current_chunk = []
            current_length = 0

            for word in chunk.split():
                word_start = current_length
                word_end = current_length + len(word)

                # Check if 'start' and 'end' are not None before comparison
                if any((start is not None and end is not None) and (start <= word_start < end or start < word_end <= end) for start, end in entity_positions):
                    current_chunk.append(word)
                    current_length += len(word) + 1
                else:
                    if current_chunk:
                        processed_chunks.append(' '.join(current_chunk))
                        current_chunk = []
                        current_length = 0

                    if len(word) > max_chunk_length:
                        sub_chunks = [word[i:i + max_chunk_length] for i in range(0, len(word), max_chunk_length)]
                        processed_chunks.extend(sub_chunks)
                    else:
                        current_chunk.append(word)
                        current_length += len(word) + 1

            if current_chunk:
                processed_chunks.append(' '.join(current_chunk))

        preprocessed_text = ' '.join(processed_chunks)

    return preprocessed_text

# Example usage
text_to_preprocess = "Desde hoy aumenta un 37,5%, pero habrá una segunda suba en febrero, cuando la tarifa pasará a $ 125. Macri impulsa los aumentos para Emova mientras despide estatales y mira desde Punta del Este. La Ciudad analiza sumarse a los aumentos mensuales dispuestos por Nación. El pasaje del subte en la Ciudad de Buenos Aires desde este viernes cuesta $ 110, un incremento de 37,5%. Es parte del primero de los dos tramos de aumentos autorizados por el Gobierno porteño el mes pasado. Se completará con una segunda suba en febrero, cuando la tarifa pasará a $ 125. La tarifa del Premetro pasó a ser de $ 38,50. Desde Subterráneos de Buenos Aires (Sbase), la firma estatal a cargo de la supervisión del subte operado por la privada Emova (Grupo Roggio) señaló en un comunicado que el aumento permitirá “seguir garantizando el mantenimiento y correcto funcionamiento del servicio"". No es lo que opinan los usuarios de la red, que viajan en condiciones de hacinamiento y con problemas estructurales; ni tampoco los metrodelegados, que rechazan las condiciones de trabajo y la contaminación por asbesto. ⚠️ Información respecto a la tarifa de Subte y Premetro pic.twitter.com/QUIwiRXGkZ — BA Subte (@basubte) January 4, 2024 El aumento del subte se suma al de los colectivos, que pasaron el 1 de enero a cobrar una tarifa mínima de $ 76,92 en el Área Metropolitana (AMBA). Los trenes del área metropolitana, por su parte, cobrarán una tarifa mínima de $ 37,38 desde el lunes 15 de enero en las líneas Roca, Belgrano Sur y Belgrano Norte; y de $ 43,38 en las líneas Mitre, Sarmiento, San Martín y Urquiza. En ambos casos, se trata de un incremento del orden del 45%. Recordemos que en los próximos días se realizará una audiencia pública para definir una nueva tarifa, con recortes de subsidios, tanto para trenes como para colectivos. A dichos recortes de subvenciones, se le sumarán incrementos mensuales de acuerdo con la inflación acumulada, según lo dispuso la Resolución 8/2023 publicada el miércoles pasado. El Gobierno Nacional invitó a otras jurisdicciones a sumarse a esta fórmula de aumento, aunque se desconoce aún si la Ciudad de Buenos Aires adherirá a la misma con los subtes. Desde el Frente de Izquierda rechazan los tarifazos. Plantean el fin de las concesiones privadas y la gestión estatal de los transportes, bajo administración de trabajadores y comités de usuarios."

# Define custom stopwords and expressions to remove
undesired_characters = ['ejemplo', '_']
custom_expressions = ['¡hola!', 'funcionará bien', "grupo de diarios américa", "recaptcha", "descargá","contactomaderomediagroupcom", "gda", "copyright", "app", "©"]

# Set remove_stopwords and remove_ner to True or False
remove_stopwords = False
remove_ner = True

# Set the maximum length for each chunk
max_chunk_length = 512

# Perform preprocessing
preprocessed_text = preprocess_text(text_to_preprocess, undesired_characters, custom_expressions, remove_stopwords, remove_ner, max_chunk_length)

# Print the result
print("Original Text:")
print(text_to_preprocess)

print("\nPreprocessed Text:")
print(preprocessed_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Original Text:
Desde hoy aumenta un 37,5%, pero habrá una segunda suba en febrero, cuando la tarifa pasará a $ 125. Macri impulsa los aumentos para Emova mientras despide estatales y mira desde Punta del Este. La Ciudad analiza sumarse a los aumentos mensuales dispuestos por Nación. El pasaje del subte en la Ciudad de Buenos Aires desde este viernes cuesta $ 110, un incremento de 37,5%. Es parte del primero de los dos tramos de aumentos autorizados por el Gobierno porteño el mes pasado. Se completará con una segunda suba en febrero, cuando la tarifa pasará a $ 125. La tarifa del Premetro pasó a ser de $ 38,50. Desde Subterráneos de Buenos Aires (Sbase), la firma estatal a cargo de la supervisión del subte operado por la privada Emova (Grupo Roggio) señaló en un comunicado que el aumento permitirá “seguir garantizando el mantenimiento y correcto funcionamiento del servicio. No es lo que opinan los usuarios de la red, que viajan en condiciones de hacinamiento y con problemas estructurale

In [36]:
# Original Text
print("Original Text:")
print(text_to_preprocess)

# Preprocessed Text with Named Entities
print("\nPreprocessed Text with Named Entities:")
print(preprocessed_text)

# Preprocessed Text without Named Entities
preprocessed_text_without_ner = preprocess_text(text_to_preprocess, remove_ner=True)
print("\nPreprocessed Text without Named Entities:")
print(preprocessed_text_without_ner)


Original Text:
Desde hoy aumenta un 37,5%, pero habrá una segunda suba en febrero, cuando la tarifa pasará a $ 125. Macri impulsa los aumentos para Emova mientras despide estatales y mira desde Punta del Este. La Ciudad analiza sumarse a los aumentos mensuales dispuestos por Nación. El pasaje del subte en la Ciudad de Buenos Aires desde este viernes cuesta $ 110, un incremento de 37,5%. Es parte del primero de los dos tramos de aumentos autorizados por el Gobierno porteño el mes pasado. Se completará con una segunda suba en febrero, cuando la tarifa pasará a $ 125. La tarifa del Premetro pasó a ser de $ 38,50. Desde Subterráneos de Buenos Aires (Sbase), la firma estatal a cargo de la supervisión del subte operado por la privada Emova (Grupo Roggio) señaló en un comunicado que el aumento permitirá “seguir garantizando el mantenimiento y correcto funcionamiento del servicio. No es lo que opinan los usuarios de la red, que viajan en condiciones de hacinamiento y con problemas estructurale

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Preprocessed Text without Named Entities:
hoy aumentar 375 segundo subir febrero tarifa pasar 125 macri impulso aumento emova mientras despedir estatal mirar punto ciudad analizar sumar él aumento mensual dispuesto nación pasaje subte ciudad buenos aires viernes costar 110 incremento 375 parte primero dos tramo aumento autorizado gobierno porteño mes pasado completar segundo subir febrero tarifa pasar 125 tarifa premetro pasar ser 3850 subterráneo buenos aires sbase firma estatal cargo supervisión subte operado privado emova grupo roggi o señalar comunicado aumento permitir seguir garantizar mantenimiento correcto funcionamiento servicio opinar usuario red viajar condición hacinamiento problema estructural tampoco metrodelegado rechazar condición trabajo contaminación asbesto información respecto tarifa subte premetro pictwittercomquiwirxgkz ba subte basubte january 4 2024 aumento subte sumar colectivo pasar 1 enero cobrar tarifa mínimo 7692 área metropolitano ambar tren áreo metropol

In [12]:
import spacy

def spanish_lemmatizer_with_pos(text):
    # Load spaCy model for Spanish
    nlp = spacy.load('es_core_news_sm')

    # Process the text using spaCy
    doc = nlp(text)

    # Extract lemmatized tokens with POS information
    lemmatized_tokens = [(token.text, token.lemma_, token.pos_) for token in doc]

    return lemmatized_tokens

# Example usage
text_to_lemmatize = "Argentina y un giro transcendental en su historia: El voto a Milei es la búsqueda de la cura para la enfermedad,https://derechadiario.com.ar/opinion/argentina-y-un-giro-transcendental-en-su-historia-el-voto-a-milei-es-la-busqueda-de-la-cura-para-la-enfermedad,laderechadiario,right_wing,Publicado en Por El diputado nacional fue el candidato más votado en las primarias y agradeció “a todos los que vienen apostando desde el 2021 en crear un proyecto liberal, con proyección nacional y que llegue a ser gobierno”.   Los comicios del pasado domingo 13 de agosto en Argentina no pasarán desapercibidos en su historia. Sucede que el candidato libertario Javier Milei dio el batacazo imponiéndose en las urnas por encima de las dos fuerzas políticas hegemónicas, Juntos por el Cambio, respaldado por la UCR, y Unión por la Patria, respaldado por el PJ.  Si bien es cierto que esto no es algo definitivo dado que se trata de elecciones primarias y la distancia entre los tres primeros candidatos no es significativa, no deja de representar un hito histórico en una sociedad víctima de una serie de mitos y construcciones sociales que afectaron severamente su moral. Argentina es un país que ha naufragado por terribles vaivenes políticos, institucionales y económicos a lo largo de su historia reciente: golpes de estado, dictaduras, dos hiperinflaciones, una guerra, crisis económicas recurrentes, etc. Estos antecedentes no son menores porque son los encargados de enmarcar los valores morales de una sociedad.  Históricamente los argentinos fueron víctima del saqueo institucionalizado donde los gobernantes los han engañado de forma sistemática y permanente. Entre los tantos ejemplos que vienen a mi mente lo constituye el Banco Central y varios impuestos que fueron creados de “forma transitoria” con la promesa de ser “eliminados” en el “corto plazo” para convertirse en definitivos conviviendo con la gente incluso hasta nuestra actualidad. Se les confiscaron a las personas sus ahorros sin piedad alguna, la pobreza y la inseguridad se volvieron moneda corriente y la libertad se convirtió en una expresión vacía de contenido. Argentina es un país donde una gran cantidad de gente no cree en la propiedad privada, pilar fundamental de la civilización. Esta última es solo una “cuestión relativa”, algo que puede prostituirse en función a las “necesidades políticas” del momento, para muchas personas. Es por ello que el argentino promedio está enfermo, ya que está infectado de mentiras y discursos que le hicieron creer que todos los males son culpa de los “inescrupulosos y codiciosos empresarios” que solo quieren “cagar a la gente”. Mientras que en realidad el Estado (ellos, los burócratas estatales) son el mesías al cual deben confiarle todo: la salud, la educación, la seguridad, la vivienda, etc.  A lo largo de generaciones se habló de esto, de la “justicia social”, caracterizada por un fuerte y marcado paternalismo estatal. La justicia social, por más “bonito” que pueda sonar en términos semánticos, no es justicia ni es social.  Esto es porque consiste en un trato desigual ante la ley, castigando la propiedad privada de quienes mejor se desempeñan en el proceso productivo para financiar la maquinaria de “derechos” convirtiendo en rehenes a quienes son esquilmados a impuestos y a quienes son “beneficiarios de los derechos” generándose así un círculo vicioso de dependencia estatal permanente.  Si hacemos un análisis profundo y racional del mensaje que se oculta detrás de la justicia social podremos concluir que es el siguiente: “como vos nunca serás capaz de pagarte la salud, la educación, la seguridad y la vivienda por vos mismo, nosotros, representantes iluminados del Estado, ente supremo y superior a vos, haremos que puedas acceder. Me encargare de dirigir tu vida por siempre porque tu incapacidad no te permitirá progresar, necesitaras siempre de nosotros”. Desde nuestros abuelos este adoctrinamiento en la religión del estado se hizo institucional “encarnandose” en el ideario popular. Hace casi un siglo que las personas no confían en sí mismas. El individuo fue degradado a la mínima expresión.  La sociedad decidió delegar sus propias responsabilidades individuales en el colectivo con la creencia falaz de que los gobernantes se encargarían de trabajar en pos del “bienestar general”. Es por lo que, con toda lógica, hablar hoy de “vouchers educativos” o “seguro de salud universal” o de “libre mercado” suena como una broma de mal gusto o “mala palabra”. Es a partir de esta lógica que la clase política lucró, lucra y seguirá lucrando hasta tanto sepamos reaccionar. Estamos ante verdadero despertar de la sociedad, donde Milei será solo un mero protagonista pasajero. Por ello, será importante que comprendamos que los verdaderos cambios no vendrán a través de Milei (ni de ningún político) en democracia, sino más bien a través de nosotros mismos, los verdaderos protagonistas, los individuos. Existe hoy en la sociedad un fenómeno disruptivo de individuos que comienzan a creer en ellos mismos, en su libertad, en su razón, en sus capacidades y que, con el tiempo, nada ni nadie les podrá arrebatar su libertad bajo ningún tipo de engaño colectivista. ¡Despertad, aún no hemos roto nuestras cadenas! Por Hernán Vionnet, para La Derecha Diario.  No, no son lo mismo: Argentina tiene la oportunidad de elegir a los mismos de siempre o apostar por algo distinto Por qué Milei: El candidato de la Argentina postergada se abrió camino en las urnas Milei en Davos: En un discurso histórico, defendió las ideas libertarias, apuntó contra la justicia social y la Agenda 2030 Fuerte recomposición de Reservas del BCRA: Se logró comprar más de US$ 4.600 millones desde el 11 de diciembre El Gobierno de Milei lanzó un operativo contra los pesqueros chinos que depredan ilegalmente al Mar argentino Milei le ahorra al Estado argentino casi 400.000 dólares viajando a Davos en un vuelo comercial con escala en Frankfurt Los fabricantes de maquinaria agrícola apuestan a la reactivación de la economía por las reformas del Gobierno de Milei La OMS y el Foro de Davos se preparan para una “Enfermedad X” que podría volver a llevar al mundo a cuarentena Publicado en Por Todas las condiciones están dadas para que el hombre más rico del mundo, que ya está mirando a la Argentina con Starlink, piense en invertir con su otra empresa: Tesla.   El triunfo de Javier Milei, las características de los recursos argentinos y el aspecto socio-económico del país indican que ha llegado una situación muy favorable para que Tesla, la mega-compañía de autos eléctricos de Elon Musk, abra una fábrica (conocidas como Gigafactory) en Argentina. El presente análisis, tiene como objetivo abordar los aspectos más relevantes de esta pregunta, y romper con una serie de mitos sobre el litio, el daño ambiental y la economía argentina. Permítame presentarme: soy Félix, un profesional con cinco años de experiencia en el sector de las Energías Renovables.  He trabajado en proyectos de escala eléctrica en Chile y proyectos de generación distribuida en Argentina. Mi función involucra la gestión de proyectos y la supervisión de asuntos relacionados con los impactos ambientales. Además, soy licenciado en Gestión Ambiental.   Argentina es un país rico en recursos naturales y tiene amplio territorio disponible para nuevos proyectos. Desde el punto de vista eléctrico, particularmente en Energías Renovables, el país posee condiciones óptimas para el desarrollo del sector.  El territorio argentino tiene un potencial significativo para el desarrollo de la Energía Solar, particularmente en las regiones andina y subandina, desde Jujuy hasta Neuquén. La Puna y la Quebrada de Humahuaca también exhiben niveles sustanciales de radiación, con una irradiancia horizontal global (GHI), que oscila entre 2400 y 2700 kWh/m2.  Además, Argentina es un participante notable en la energía eólica, ubicándose entre los seis primeros países con patrones eólicos consistentes, con un factor de capacidad promedio anual del 20%. En consecuencia, una potencial Gigafactory en el país podría utilizar y suministrar fácilmente energía limpia a todo el complejo. Además, existe potencial para establecer un Parque Solar para generar energía para el sistema eléctrico argentino.  En el ámbito del desarrollo de baterías eléctricas, el recurso más importante es el litio, del cual Argentina ocupa el segundo puesto con reservas mundiales con 98 millones de toneladas de carbonato de litio (LCE). Aún con una severa subinversión, es el cuarto productor a nivel mundial, después de Australia, Chile y China, con una capacidad de producción anual de 37.500 toneladas. El triángulo del litio -formado por Argentina, Bolivia y Chile- posee alrededor del 65% de los recursos mundiales de litio. Esto no sólo ha atraído la atención de las empresas mundiales de vehículos eléctricos, sino que también ha estimulado las inversiones en Argentina. Ejemplos notables incluyen el Grupo Chery, que planea construir una planta en el país para producir vehículos eléctricos y baterías de litio.  Asimismo, Stellantis, producto de la fusión de FCA (Fiat-Chrysler) y PSA (Peugeot-Citroën), anunció una inversión de 90 millones de dólares para adquirir una participación del 19,9% en la empresa argentina Lithium & Energy (ALE). Este movimiento estratégico tiene como objetivo asegurar un suministro anual de 15.000 toneladas de litio, facilitando su uso en baterías de vehículos eléctricos.  Establecer una Gigafactory cerca de una de las reservas de litio más grandes del mundo podría generar algunos de los costos de producción más bajos para vehículos eléctricos a nivel mundial, especialmente considerando los abundantes recursos solares en la región. Además, la proximidad de Argentina a la Cordillera de los Andes contribuye a tener uno de los sectores mineros más ricos del mundo. Entre los recursos minerales del país destacan nitratos, yodo, sal común, sales de litio y potasio, boratos, áridos, arcillas, calizas, yeso, cuarzo, barita, caolín, feldespato, talco, diatomita, bentonita, dolomita, guano, apatita, zeolitas y perlita. También es fundamental resaltar que el precio del litio alcanza consistentemente máximos históricos año tras año. Así, Tesla no sólo podría establecer su Gigafactory en Argentina, sino también involucrarse en el sector minero para reducir los costos de producción.   Habiendo trabajado personalmente en el desarrollo de proyectos en Chile y Argentina y habiendo discutido permisos ambientales con colegas, puedo dar fe de que obtener la aprobación para la construcción no es tan burocrático como en algunos países vecinos, como en Chile.  La evaluación de impacto ambiental y el desarrollo del proyecto podrían procesarse en un período relativamente corto para iniciar el proyecto. Se requerirán estudios ambientales con consultores locales según sea necesario tanto para el proyecto minero como para la Gigafactory, pero la perspectiva es muy positiva. En conclusión, creo que Argentina está estratégicamente posicionada por su cercanía a recursos, lo que la convierte en uno de los países donde los costos de construcción de vehículos eléctricos son más económicos de la región. Los recursos solares podrían generar energía para la minería, la Gigafactory, y el excedente de energía podría venderse a la red.  La proximidad de la Gigafactory a su fuente de litio podría reducir significativamente los costos de litio de Tesla para la producción de vehículos, mejorando la eficiencia en comparación con la competencia. Además, la disponibilidad de diversos recursos en la región andina y la facilidad para obtener permisos de construcción hacen que Argentina sea ambientalmente ideal para tales emprendimientos.  Históricamente, Argentina fue uno de los países más innovadores de la región durante tiempos de mayor libertad económica. A continuación se detallan algunos de los inventos más importantes creados por argentinos, que demuestran la capacidad y talento para el trabajo en nuestro país: 1. Amortiguador hidroneumático (Francisco Abolio, 1930). 2. Bolígrafo. 3. Bypass coronario de René Favaloro. 4. Sistema de autobuses públicos (Colectivo). 5. Corazón artificial (Domingo Liotta y Denton Cooley, 1969). 6. Método Galli Mainini para el diagnóstico precoz del embarazo. 7. Dibujos animados. 8. Encendedor piezoeléctrico Magiclick de Hugo Kogan (1969). 9. Helicóptero de Pescara. 10. Identificación dactilar de Vucetich. 11. Instrumentos médicos de Favaloro, Finochietto y otros. 12. Instrumento de navegación nocturna para aeronaves (Almonacid, 1925). 13. Jeringa desechable (Carlos Arcuzzi, 1979). 14. Marcapasos cardíaco (Francisco de Pedro, 1979). 15. Membrana de colágeno curativa (Celia Mohadeb, 1999). 16. Método de conservación de sangre con citrato de sodio por Agote. 17. Motor de combustión interna “Péndulo” (Eduardo Taurozzi, 1970). El talento argentino en innovación y persistencia siempre se ha destacado en el tiempo. Lamentablemente, los problemas económicos y las excesivas regulaciones han reducido la capacidad de innovación, pero las capacidades de los argentinos siguen siendo evidentes, demostrando dedicación en todos sus emprendimientos.  A medida que Argentina comience a crecer económicamente, podemos esperar ver mentes brillantes brillar una vez más en nuestro país. A pesar de soportar años de desafíos económicos, muchos argentinos han brillado tanto en el país como globalmente por sus capacidades, y muchos otros aún tienen que demostrar su potencial.   Pero lo más importante, el desarrollo de la Gigafactory generará oportunidades de empleo de alta calidad en la zona, ya que será necesario emplear una gran parte del personal utilizando mano de obra local, además de ingenieros, geólogos, y expertos que conozcan la zona. Esto también tendrá un impacto positivo en los servicios de la zona, como alojamiento, sanidad, luz, ya que deberán ser desarrollados como apoyo a la mega-fábrica. Respecto a la locación, la región norte del país cuenta con extensos terrenos, minimizando la posibilidad de impactar negativamente las actividades sociales. Las comunidades indígenas más conflictivas se encuentran en en el Sur del país. Optar por el desarrollo en la región Norte, donde la Gigafactory y las actividades mineras se consideran óptimas, minimiza la probabilidad de encontrar problemas con las comunidades locales. Debajo se adjunta una imagen que muestra la distribución geográfica de las comunidades indígenas en Argentina, para dar sustento a los puntos previamente mencionados:  Sin duda, uno de los factores más cruciales a destacar es la situación económica de Argentina. El país ha experimentado intervenciones gubernamentales prolongadas, interfiriendo con el sector privado y provocando un empobrecimiento cada vez mayor debido a la falta de inversión y generación de riqueza. Las medidas de justicia social, destinadas a redistribuir la riqueza de los productores a los pobres, han demostrado conseguir el efecto inverso en Argentina, lo que ha resultado en que el 50% de la población viva en la pobreza, con 6 de cada 10 niños enfrentando inseguridad alimentaria. Estas políticas sociales no sólo no han logrado beneficiar a la sociedad como se esperaba, sino que también han impactado negativamente al sector privado. Hace pocas semanas, Javier Milei, el primer economista libertario en asumir la Presidencia, señaló un posible cambio en el futuro de Argentina con reformas liberales.  Aquí hay una breve descripción de las reformas económicas propuestas por Milei 1. Reforma monetaria: La adopción del dólar estadounidense como moneda oficial significa un cambio monumental en el panorama económico de Argentina. Al vincular la moneda a un estándar reconocido mundialmente, Milei pretende infundir confianza a los inversores, mitigar las preocupaciones sobre las fluctuaciones monetarias y fomentar un entorno financiero más estable. 2. Reducción de la intervención gubernamental: La propuesta de Milei de minimizar las funciones del Estado se alinea con los principios económicos liberales. Esto incluye simplificar los procesos regulatorios, reducir los obstáculos burocráticos y crear una atmósfera más favorable a los negocios para las corporaciones extranjeras. 3. Impacto potencial en las inversiones extranjeras: Si se implementan efectivamente, estas políticas económicas propuestas pueden atraer a inversionistas extranjeros que buscan estabilidad y oportunidades de crecimiento. El énfasis en soluciones impulsadas por el mercado y la reducción de la interferencia gubernamental podrían posicionar a Argentina como un destino atractivo para empresas, como Tesla, que buscan expandir sus operaciones.  4. Desregulación de la economía: A solo una semana de la presidencia de Milei, se están realizando esfuerzos para implementar importantes reformas de desregulación económica. Se han eliminado numerosos ministerios redundantes y se han derogado más de 300 leyes que perjudicaban a la economía. Si bien invertir en Argentina todavía puede plantear riesgos, con el ambicioso plan de Javier Milei y el rápido inicio de reformas, podríamos estar ante el surgimiento de una nueva potencia económica en la región. Ser uno de los primeros socios del gobierno en un período tan transformador podría ser una medida estratégicamente inteligente, que podría generar beneficios y oportunidades sustanciales para quienes participan.  La sinergia potencial de factores sociales, ambientales y económicos en Argentina crea una narrativa convincente para que Tesla explore el establecimiento de una Gigafactory y una operación minera en el Norte argentino.  Si bien los desafíos persisten, la perspectiva de alinearse con un panorama político y económico cambiante, aprovechar recursos abundantes y contribuir al desarrollo sostenible posiciona a Argentina como una frontera prometedora para la expansión estratégica de Tesla.  Publicado en Por El fallido operativo de la Policía de la Ciudad para acatar el Protocolo Antipiquetes en la marcha de este 20 de diciembre vuelve a poner en tela de juicio la necesidad de que la Policía Federal intervenga en la dispersión de manifestantes que atacan el microcentro porteño.   En ocasión de la primera marcha piquetera de protesta contra el gobierno del Presidente Javier Milei, resulta de mucha utilidad repasar algunos conceptos fundantes que hacen a la siempre conflictuada noción de “orden público” en el territorio de la Ciudad Autónoma de Buenos Aires, Capital de la República Argentina. No es sorpresivo que el accionar piquetero tienda a concentrarse en la Capital. Después de todo, el objetivo final de las organizaciones que marchan es condicionar al poder político a fin de obtener privilegios y beneficios para las supuestas situaciones de vulnerabilidad que revistan. Párrafo aparte merecería el destino de los fondos que mensualmente perciben de parte de los organismos del Estado Nacional, pero dejando de lado ese detalle es que nos remitimos a la cotidianidad de los trabajadores que deben transitar por las zonas afectadas por las marchas piqueteras. En la Ciudad de Buenos Aires circulan más de diez millones de personas por día por motivos laborales. Gran parte de esta cifra, se ve obligada a transitar por el llamado “microcentro”, zona geográfica predilecta para el desarrollo de actividades económicas, y por eso también, la favorita para los piqueteros.  Durante más de veinte años la sociedad se ha acostumbrado al sometimiento que las organizaciones sociales ejercen sobre el pueblo trabajador: desde cambiar el recorrido diario, llegar más tarde al trabajo o inclusive la suspensión de la jornada laboral en días de marcha, son solo algunas de las cientos de consecuencias que las movilizaciones producen.  Sin embargo, y pese a la ilegalidad manifiesta del accionar piquetero, el poder político no había abordado la tortuosa temática por un mismo motivo que fue denominador común sin importar el color partidario del gobernante de turno: la corrección política. Este miércoles, finalmente, el flamante Presidente Milei decidió que era hora de cortar con los intentos extorsivos. En el marco de un operativo de seguridad sin precedentes, la Gendarmería Nacional, Policía Federal, Prefectura Naval, Policía de Seguridad Aeroportuaria y el Servicio Penitenciario Federal blindaron las calles aledañas a Plaza de Mayo, no sin previamente cerciorarse de disponer delegaciones tácticas en estaciones de tren, accesos automotores, peajes y en localidades del Gran Buenos Aires donde se desactivaron células piqueteras durante toda la noche.  El operativo de seguridad tenía como objeto impedir que los piqueteros cortasen la circulación en las principales arterias de la Ciudad de Buenos Aires y por ende permitir a todos los ciudadanos ejercer su derecho a la libertad de movimiento. Sí bien los lineamientos generales del plan fueron ampliamente cumplidos generando un “antes y un después” en la manera que el Gobierno Federal aborda la temática de los cortes piqueteros, corresponde hacer una desafortunada observación: el limitado accionar de la policía de la Ciudad de Buenos Aires.  Con el afán de estar a la altura de las circunstancias, el Gobierno de la Ciudad de Buenos Aires (GCBA) ordenó un operativo paralelo presuntamente coordinado con el Comando Unificado de las Fuerzas Federales.  El Centro de Monitoreo Urbano (CMU) de la Policía de la Ciudad fue el escenario donde montó su Cuartel General el Ministro de Seguridad porteño Waldo Wolff quien, desde tempranas horas de la mañana se encontraba en comunicación con Patricia Bullrich.  Durante los momentos previos al apogeo de la marcha alrededor de las 16:00 todo parecía marchar coordinadamente: las Fuerzas Federales y la Policía de la Ciudad conducían ordenadamente a los manifestantes hacia Plaza de Mayo, mayoritariamente por la vereda. Empero, al aumentar el grosor de las columnas, los manifestantes comenzaron a tensionar la situación. En primera instancia, actuó la Policía de la Ciudad en Diagonal Sur mientras su Jefe y Viceministro de Seguridad porteño, Diego Kravetz (quien se encontraba en el teatro de operaciones), alegaba que “no necesitaban del apoyo de las Fuerzas Federales”.  Tal fue así que cuando los manifestantes decidieron avanzar sobre la Policía capitalina, no encontraron mayor resistencia y las prolijas líneas de efectivos fueron fácilmente rebasadas con empujones e insultos mediante. Por este motivo, se ordenó a la Policía Federal hacerse cargo luego de que las columnas -que ya copaban el ancho completo de la Diagonal- pasaron la estatua ecuestre del General Julio A. Roca ubicada en la intersección con la calle Perú. Una vez restablecido el orden y conducidas las poco nutridas columnas manifestantes a Plaza de Mayo por parte de las Fuerzas Federales, los medios de comunicación pudieron averiguar que el Viceministro Kravetz había dialogado con el líder piquetero Eduardo Belliboni, autorizándolo a avanzar sobre la totalidad del ancho de las calles, produciendo indefectiblemente el corte total.  Más allá del intento de connivencia de Kravetz con Belliboni -lo cual coloca en una muy poco confortable situación al gobierno de Jorge Macri- corresponde remitir a la cuestión neurálgica de estos problemas, que es la superposición de facultades entre las Fuerzas Federales y la Policía de la Ciudad. En 1995, un año después del otorgamiento de autonomía a la Ciudad de Buenos Aires por parte de la reformada Constitución Nacional de 1994, el Congreso de la Nación sancionaba la popularmente denominada “Ley Cafiero”, o “Ley que garantiza los intereses del Estado Nacional en la Ciudad de Buenos Aires”.  Dicha pieza normativa impedía al Gobierno de la Ciudad de Buenos Aires a tener una policía propia puesto que disponía que la seguridad armada fuera garantizada por la Policía Federal en el territorio de CABA mientras esta sea la Capital de la República. Tras largos debates y pujas de poder, el Congreso modificó la Ley Cafiero en 2007, mediante la Ley 26.288 para permitirle a CABA montar su propio cuerpo policial y establecer el marco de coexistencia con la Policía Federal. Con la fundación de la Policía de la Ciudad en 2017 luego del traspaso de la Superintendencia de Seguridad Metropolitana de la PFA, CABA obtuvo finalmente su tan deseado brazo armado. Sin embargo, la evidencia empírica arroja que en cuanto al mantenimiento del orden público se refiere, la Policía de la Ciudad tiene importantes oportunidades de mejoras respecto de lo que fue la gestión de la PFA.  El problema principal radica en que se trata de una policía heterogénea, dado que sus miembros provienen de diferentes fuerzas y nunca se logró el aseguramiento de un criterio único de tropa y formación. En los tiempos que se avecinan, donde la resistencia anti-democrática al cambio que necesita nuestro país puede volverse cada vez más violenta, es indispensable contar con un criterio unificado a fines de hacer frente a las amenazas y asegurar la vigencia plena del orden público y del Estado de Derecho.  La Ciudad de Buenos Aires debe estar a la altura de las circunstancias o en su defecto, quizás la alternativa sea volver a modificar la Ley Cafiero a fines de aclarar que las operaciones de establecimiento del orden sean únicamente desarrolladas por el Estado Nacional y sus Fuerzas Federales siempre que el Gobierno Nacional sea el foco de la concentración. Sea del modo que fuere, la Argentina entró en un cambio de época irreversible. Seremos testigos de cómo cada responsable institucional se adapta a las circunstancias.  Publicado en Por Dos estudiantes han tomado la iniciativa de juntar distintos comentarios que jóvenes de todo el país le escribieron al presidente electo Javier Milei para compilar una carta histórica que aquí queda presentada.   A los 7 dias del mes de diciembre de 2023 Señor Javier Gerardo Milei. Presidente Electo de la Nación. Desde las tierras de Chaco y Mendoza, Máximo Morassutti y Lionel Schröder, dos estudiantes de 14 y 17 años, hemos tomado la iniciativa de escribirle esta carta.  Gracias a la colaboración de muchísimas personas, un grupo diverso de jóvenes de todas las provincias del país se une con entusiasmo para expresar apoyo tras su histórica victoria.  Este triunfo representa para nosotros la ruptura de las cadenas que vienen limitando nuestro potencial, con políticas intervencionistas de las cuales. sabemos. Solo se benefician unos pocos. Nos sentimos profundamente identificados con sus ideas y creemos que, bajo su liderazgo, todas las regiones de nuestra nación podrán florecer en un ambiente de libertad, justicia y prosperidad compartida. “Oíd el ruido de rotas cadenas”: un eco del himno que resuena en todos los rincones. López y Planes escribió en 1812 aquel fragmento posiblemente seguro de hacerlo, pero no consciente de que perduraría de manera tan icónica hasta los tiempos que corren… Hoy, consideramos momento de renombrarlo con su elección, entendiendo que nos encontramos frente a la apertura de un nuevo capítulo en nuestra historia: la era liberal.  A continuación, procederemos a presentar llenos de emoción los mensajes enviados por cada uno de los voluntarios: Abi Franzutti (IG: @abifranzutti) – 17 años – Provincia de Córdoba Sr. Presidente Javier Milei. Me complace presentar a usted las más cordiales felicitaciones en nombre de la provincia de Córdoba y en el mío propio, con motivo de su elección como próximo presidente de la República de Argentina. En esta propicia ocasión nos acercamos para formular nuestros mejores deseos de que la gestión de gobierno que usted habrá de encabezar en este hermoso país, haga importantes contribuciones al fortalecimiento de la democracia, al mantenimiento de la paz y la solidaridad en el pueblo argentino, y a un mayor desarrollo económico que propenda al bienestar general. Agregando a la juventud de LLA, le agradecemos por su apoyo que nos brindó en campañas electorales, y nuevamente agradecerle por todo su arduo esfuerzo, trabajo y desempeño para llegar a dónde hoy usted, sr. Presidente, está. Asimismo, Córdoba confía en sus propuestas y esperamos que fielmente así se impulsen en todo nuestro territorio. Con reiteración de nuestras congratulaciones le saluda, atentamente, Abi Franzutti.  Benjamin Gosen (X: @GOSEN_LIBERAL) – 16 años – Provincia de Tucumán Estimado Presidente Electo, Desde San Miguel de Tucumán, brindo todo mi apoyo hacia usted. Desde el interior, confiamos en que pueda poner de pie a Argentina y posicionarnos en el lugar que nos merecemos. Uno de los principales problemas de hoy en día es la inseguridad; por tanto, confiamos plenamente en su compañera de fórmula, Victoria Villarruel, en aquella área. Solo puedo decir a nuestro presidente y vicepresidente electos: ¡Muchas gracias por volver a traernos la esperanza a este hermoso país!. ¡Viva La Libertad! Ezequiel Asenjo (X: @AsenjoEzequiel) – 20 años – Provincia de Chubut Estimado presidente Javier Milei: Quiero expresar mi profundo agradecimiento por su incansable lucha en contra de la corrupción política. Su valentía y compromiso son un ejemplo para todos nosotros. Gracias por defender nuestros derechos y principios con vehemencia.  Sería interesante escuchar las opiniones de Javier Milei sobre los sindicatos y la producción de petróleo en Chubut y la Patagonia, considerando su relevancia en la economía latinoamericana. Sus perspectivas podrían aportar nuevas ideas y enfoques para abordar estos temas.  Gracias Javier milei por devolver la confianza a la gente, espero que le vaya muy bien en su mandato. Facundo Albiñana (X: @Facundo_AlbZ) – 17 años – Provincia de San Juan Estimado Javier Milei: Los jóvenes liberales de San Juan estamos entusiasmados con la vuelta de una Argentina próspera. El camino a seguir implica valentía, paciencia y perseverancia, pero estamos dispuestos a afrontar los desafíos, ya que como decía nuestro prócer Sarmiento, “Si no existieran dificultades, no habría éxitos”. No nos queda más que agradecer el trabajo de tantos años del presidente Javier Milei, en divulgar las ideas de la libertad que hicieron despertar la esperanza en un futuro mejor a la mayoría de los jóvenes de este país.  Gonzalo P. Aguilar Pouilloux (IG: @gonza.agp) – 19 años – Tierra del Fuego UNA ARGENTINA CON ESPERANZAS. Argentina, patria mía, pueblo que pude haber pensado, como tantos otros compañeros, que no podía cambiar, parecíamos destinados al fracaso, pero hace unos días hubo un domingo, ese día histórico donde el liberalismo y la derecha se impusieron ante el avance del totalitarismo, donde el pueblo se puso de pie, y gritando “¡LIBERTAD!” dijo basta a décadas de decadencia; pero esto hay que construirlo, pueblo mío, queda mucho por hacer, mucho por luchar, mucho por ganar, queda confiar en que todo saldrá bien, pero guardo mis grandes esperanzas en que el león seguirá rugiendo. ¿Si tanto hemos avanzando en la batalla cultural por la libertad, porque no podíamos ganarla totalmente? Las fuerzas se han unido en busca de una argentina libre, con orden y donde la gente luche por su vida, y respete la del otro, donde el delincuente pague el ser delincuente, aún si el delincuente es el político. Las derechas del mundo avanzan ante la caída del muro progresista de la censura y la corrección, el pueblo debe mirar a su alrededor, debe mirar a Venezuela, a Cuba, mirar las contradicciones progresistas, mirar su apoyo a delincuentes, terroristas y degradaciones morales, y ver que algo está claro, la lucha es por la moral, la lucha es por la ética, es por la patria, es por las instituciones naturales, medios entre el individuo y el Estado, esas que hay que defender y fortalecer. El hecho es luchar por la libertad, pero esta como medio, porque siempre será un excelente caballo, pero hay que tener hacia donde cabalgar, y eso es lo que venimos a proponer en esta nueva esperanza, un camino, que no pierdas lo que te queda, y que recuperes lo que perdiste, porque si ya te quitaron todo, y te quisieron dejar en la miseria, ahora nos levantamos para decir que la miseria son ellos, y que la libertad no la cederemos. Ahora queda construir la libertad, mis patriotas, sin fanatismos, y sin miedo, porque al totalitario se lo lucha, allí no habrá tolerancia soportable. ¡Viva la libertad!   Guillermo Esteban Romeo (X: @guillernoeromeo) – 20 años – CABA (Buenos Aires) “Es un honor representar en este mensaje las palabras que los jóvenes de CABA tienen para dirigirse a nuestro Presidente Electo Javier Milei.  A lo largo de los años, desde el 2021, hemos experimentado una gran alegría por su participación en la política. Desde su escaño en el Congreso, representando a los ciudadanos, hemos observado en usted la verdadera vocación de servicio a la patria. Nosotros, los jóvenes, formamos parte de un acontecimiento histórico que marcará un antes y un después. Desde las diversas agrupaciones y grupos presentes en la ciudad, hemos trabajado incansablemente para inculcar en los porteños las ideas que condujeron a que usted sea el Presidente en la actualidad. En mi caso, fue a través de la Agrupación Enlaces, donde dedicamos esfuerzos a la campaña tanto de usted como de Ramiro Marra. Queremos expresar nuestro total agradecimiento por lo que ha realizado por el país y por lo que está llevando a cabo. Le agradecemos por su humildad, coraje y, sobre todo, por haberse enfrentado valientemente a un sistema que estaba en su contra. Desde nuestro compromiso en la militancia, nos comprometemos a continuar librando la batalla cultural que ha resultado victoriosa. Nos comprometemos a defender las ideas de la libertad hasta el final de nuestras vidas, porque como dijo nuestro referente Alberdi, “No hay libertad en un hombre donde su seguridad, su vida y sus bienes están a merced del capricho de un mandatario”. Gracias por liberarnos del estado opresor y por asumir la gran responsabilidad que lleva hoy en día. Enviamos un saludo en representación de todos los jóvenes de la Ciudad Autónoma de Buenos Aires.” Saludos cordiales, Guillermo Esteban Romeo.  Javier Arce (X: @XaviArce20) – 23 años – Provincia de Misiones Estimado Pdte. Javier Milei Mi nombre es Javier Arce, y me dirijo a usted para expresarle mi más sincero apoyo y agradecimiento por su candidatura a Presidente de la Nación. Como usted sabe, Argentina atraviesa una profunda crisis económica y social. Los últimos años han estado marcados por el populismo y la incompetencia, que han llevado al país a la ruina. Usted es la única esperanza que tenemos de salir de esta situación. Sus propuestas, basadas en la libertad económica y el respeto a la propiedad privada, son las únicas que pueden llevar a Argentina al progreso.  Aprecio su valentía y honestidad: eso es lo que necesitamos en un líder. Estoy seguro de que, con su ayuda, Argentina podrá volver a ser un país próspero y libre. Atentamente: Javier Arce  Javier Pérez Coy (X: @Coy_javito) – 20 años – Provincia de La Rioja Estimado Presidente Electo Milei. Le escribo desde Chilecito, La Rioja, para expresarle algunas inquietudes que afectan a mi barrio y a nuestra comunidad. Actualmente, vivimos en condiciones precarias, con sueldos bajos y calles sin asfaltar. Las escuelas públicas carecen de recursos, y la situación no parece mejorar. El intendente Rodrigo Brizuela Idoria, a pesar de sus discursos, no ha llevado a cabo obras significativas en nuestro barrio. Rodrigo Quintela, por otro lado, ha centrado sus esfuerzos en un hospital en construcción desde hace mucho tiempo, pero aún sin completar. En cuanto a la seguridad, aunque no hay mucha criminalidad, la presencia de drogas, a veces adulteradas, es preocupante. Además, la corrupción policial, con la confiscación y subasta de vehículos, es un problema persistente. En este contexto, veo en su liderazgo una oportunidad para el cambio. Su enfoque basado en ideas y voluntarismo político es lo que nuestra provincia necesita. Carolina Moreno y Martín Menén también son figuras prometedoras que se han conectado directamente con la comunidad. Espero que su gestión traiga mejoras tangibles a nuestra provincia y municipios. Confío en que su compromiso marcará la diferencia. Le agradezco su atención y le deseo éxito en su presidencia. Atentamente.  Javier Pérez Coy  Jazmín Venini (IG: @jazmin_venini) – 21 años – Provincia de Río Negro “En este día tan especial, en representación de los jóvenes de la hermosa provincia de Río Negro, desde la pintoresca ciudad de Bariloche, quiero expresar mi más sincero agradecimiento a Javier Milei por liderar este nuevo gobierno que trae consigo un proyecto de transformación y libertad, tan esperado por todos nosotros.  Es un honor para mí, siendo una joven barilochense de 21 años, ver cómo nuestro país avanza hacia un horizonte de transformación donde los jóvenes de Río Negro nos vemos comprometidos con las ideas del liberalismo y nos sentimos representados por su visión de un futuro más libre y próspero.  Además, como mujer, encuentro inspiración en ver a la vicepresidente Victoria Villaruel a su lado, así como a la líder de La Libertad Avanza en Río Negro, Lorena Villaverde. La presencia de ambas nos llena de esperanza y nos demuestra que la participación activa de las mujeres en la política es fundamental para construir una sociedad más justa.  Compartimos su visión de mejorar la economía, impulsar el cumplimiento de la justicia y hacer frente a las usurpaciones de comunidades falsas indígenas en el territorio nacional. Estamos ansiosos por ser parte de un gobierno que defienda la propiedad privada, el libre mercado y un crecimiento hacia la Argentina de la gente de bien que con su esfuerzo pueda salir adelante.  Admiramos su compromiso con la juventud y su deseo de permitirnos tener una voz y voto en la construcción del futuro de nuestro amado país. Creemos que la participación activa de los jóvenes es clave para lograr una sociedad más dinámica y justa, y estamos emocionados por la oportunidad de contribuir a ese cambio positivo.  Finalmente, agradecemos la posibilidad de tener roles fundamentales dentro del gobierno, lo cual nos brinda la oportunidad de ser agentes de cambio en esta nueva etapa de nuestra historia. Estamos listos para trabajar codo a codo con usted y su equipo para construir un futuro lleno de oportunidades para los argentinos.” Con esperanza y compromiso, agradeciendo esta oportunidad, Jazmin Venini. Líder de la agrupación nacional “Leones Sueltos”.  Jeremías Cares (X: @JrsCares) – 18 años – Provincia de Neuquén Señor Javier Gerardo Milei. Me dirijo respetuosamente para agradecer su gran influencia en mí y en muchos jóvenes del país. En 2020, en la pandemia, descubrí la importancia de la libertad gracias a usted. No solo aprendí sobre economía sino también sobre lo mal que le hizo el Kirchnerismo a Argentina.  El día del balotaje, me alegré demasiado. A pesar de tener conflictos con mis pares por defender las ideas de la libertad, prometí que si perdías me iba del país, pero ganaste y hoy me toca quedarme y luchar por el país que tanto amo. Juan Ignacio Climent (IG: @_juanicliment_) – 21 años – Provincia de Santiago del Estero Estimado Presidente Electo Sr. Javier Milei, Es un honor dirigirme a usted en calidad de ciudadano en conjunto con el resto de jóvenes para expresar mis más sinceras felicitaciones por su elección como Presidente de la República Argentina. Este triunfo representa una luz de esperanza para todos los argentinos que queremos un país distinto. Admiro su compromiso por levantar a la Argentina y colocarla en el lugar que merece estar. Después de años de miseria y decadencia política, es momento de que aquello ocurra; y por esto depositamos nuestra confianza en usted, no nos defraude como nos defraudaron sus antecesores.  Asimismo, quiero resaltar la importancia del diálogo y el consenso en nuestro país, que no es susceptible de homogeneizar. La diversidad, las diferencias y la tolerancia son necesarias para el progreso pacífico y democrático de nuestra Nación. Basta de grieta. Finalmente, y con pleno respeto por la responsabilidad que ha asumido, como santiagueño siento necesario pedirle que durante los próximos años de su gestión tenga presente a mi provincia y a su desarrollo económico y social, ya que aquí la libertad siempre se pone en duda y cuestión. Sin más que agregar, le deseo el mayor de los éxitos en su gobierno y le ofrezco mi apoyo desde mi provincia para lograr los cambios positivos que el país necesita. Estoy convencido de que así será. Atentamente, Juan Ignacio Climent  Lionel Schröder (IG: @lioschroder) – 17 años – Provincia del Chaco “Tengo 17 años, siempre viví en crisis, siempre escucho en las noticias lo corruptos que son los políticos, el hambre que pasa la gente, la inflación que nos hace más pobres, ministros negociando con el fondo, causas de corrupción, muertes, robos, argentinos rebuscándosela para salir adelante.  Eso vi TODA mi vida: argentinos rebuscándosela… Gente de bien, gente con valores, gente con cultura, con trabajos, con sueños, con metas, con hijos.  Todos estos argentinos tienen algo que los limita y limitó siempre: un aparato ESPANTOSO, el monopolio de la fuerza, que con excusas tontas e inocentes avanzan contra la propiedad, contra la libertad de los argentinos, este aparato cínico tiene nombre; el ESTADO.  Desde que nacemos escuchamos esta palabra y nos parece normal. Nos parece normal que el político de turno regule precios, las exportaciones e importaciones. Que se meta en cómo se debe educar a los niños, en que debemos pensar, hasta en qué moneda podemos o no comerciar.  A muchas personas les parecía normal que el estado tenga el poder de encerrarte en tu casa sin poder trabajar, estudiar, caminar, visitar amigos, familia, etc. Con la excusa de una enfermedad, nos privaron de toda libertad. Ahí en el medio de ese momento, donde el estado demostraba realmente lo que era capaz de hacer (mantener a personas presas dentro de sus casas), conocí al que es hoy el Presidente de la República Argentina, que con euforia, pasión y efusividad, defendía ideas que no estábamos acostumbrados a escuchar. Poco  a poco este hombre fue despertando una cuestión nueva en mí (y en muchos más). Una pasión, un sentimiento, un nuevo sueño: VER A LA ARGENTINA LIBRE. Ese Lionel de 14­-15 años entendió que tenía que luchar y tenía que ser la voz, para sacar al maldito estado del medio y que la gente abra los ojos, que podamos prosperar y no tener que migrar para progresar. Hoy. 3 años después, estoy más que agradecido con Javier Milei porque es el hombre que sembró las ideas de la libertad en millones de personas, y junto a miles de jóvenes queremos cosechar esa gran siembra para poder vivir libres con paz, y no olvidarnos nunca de que UN HOMBRE y MILES DE JÓVENES hicieron la Argentina grande de nuevo. Yo hoy puedo estar emocionado, pero el Lionel de 14 años se que lo estaría más, Le doy gracias a Dios por su gran amor y por haber iluminado a Javier, le seguiré pidiendo a Dios que le de sabiduría y que sea el instrumento para levantar a la Argentina. Lucas Ovarzo (IG: @loyarzo901) – 22 años – Provincia de Santa Cruz FORJANDO UN FUTURO. “En medio de desafíos y anhelos compartidos, los jóvenes de Santa Cruz encontramos en Javier Milei, una fuente de esperanza para un futuro vibrante. Su enfoque audaz y compromiso con los principios liberales resuenan profundamente en nuestra generación, que anhela un cambio significativo y progresivo. En el corazón de Argentina y Santa Cruz, vislumbramos la posibilidad de un renacimiento con Milei. El horizonte se ilumina con la promesa de un gobierno transparente, eficiente y comprometido con la prosperidad económica. Anhelamos una provincia donde el talento y la creatividad de los jóvenes se liberen de restricciones innecesarias. Visualizamos un país donde la libertad individual sea respetada, la innovación sea aplaudida y cada ciudadano tenga la oportunidad de contribuir al progreso nacional. Los niños, adolescentes, jóvenes y adultos de Santa Cruz y Argentina nos destacamos al elegir a Milei como la voz que llevará nuestras aspiraciones al escenario nacional. Confiamos en que, juntos, construiremos una Argentina donde la esperanza florezca y las oportunidades se multipliquen para cada generación venidera. Esta elección es nuestra apuesta por un mañana lleno de promesas. En Santa Cruz, se le puso fin al Kirchnerismo caracterizado por la corrupción. Este momento marca un hito significativo en la historia de la provincia, señalando el inicio de una nueva fase centrada en la transparencia, la integridad y el avance.”  Una Argentina es imposible con los mismos de siempre. VLLC! Lucas Oyarzo, referente de Pibes Libertarios S.C  Manuel Agustín Galdame (X: @Manuboy28) – 23 años – Provincia de San Luis Estimado Javier Gerardo Milei. Desde San Luis, festejamos su triunfo con esperanza.  El cambio que su elección representa es muy positivo para Argentina. Tu victoria nos llena de optimismo, confiamos en que vas a liderar al país hacia un futuro próspero y brillante, dónde recuperemos el camino y los valores que nunca debimos dejar atrás. Vamos juntos a un ciclo de crecimiento, libertad y prosperidad. El camino no será fácil, pero “no hay noche que no haya sido derrotada por el amanecer.” ¡VIVA LA LIBERTAD! Marco Guerrero (X: @PrimoG_333) – 31 años – Provincia de Salta Sr, Presidente.   Salta está contigo. Somos una provincia que sufre la pobreza, la falta de vivienda, la precariedad sanitaria y el atraso productivo. Queremos ser libres de la opresión estatal, la corrupción y el saqueo.  Confiamos en usted y en nuestra vicepresidenta, que nos representan con valentía. Salta será grande sólo si es liberal, como usted nos enseña.  Marcos Zapata (X: @Markitos8221) – 17 años – Provincia de Jujuy Estimado Javier Milei: Mis mayores apoyos y buenos deseos hacia usted y su vicepresidente Victoria Villarruel. Fue una campaña desafiante, pero su determinación y convicción generó un cambio significativo en la elección de los argentinos, obteniendo el apoyo de la mayoría.  Mi involucramiento como fiscal electoral me brindó esperanza en un futuro próspero para Argentina, aprendiendo mucho de usted y defendiendo sus votos en contra del fraude. Su ascenso representa sin dudas un cambio de paradigma. Desde Jujuy, veo oportunidades para un mejor futuro económico con líderes liberales. Espero que bajo su liderazgo, mi provincia reciba acompañamiento e impulso para su crecimiento. Su triunfo, despierta en mí emociones de esperanza y compromiso con un futuro prometedor. Confío en que liderará el cambio hacia la libertad y el progreso. Mateo Beconi (X: @MateoBeconi) – 19 años – Provincia de La Pampa Estimado Sr Javier Presidente de la República Argentina: En este año nada me hizo más feliz que tu victoria en las últimas elecciones. Fue una sensación increíble de alivio haber presenciado este triunfo, y por eso es de mi agrado enviarle mis felicitaciones a través de esta carta. Quería comentarle que espero con muchas ansias su asunción para con sus nuevas políticas levantar proyectos propios, pudiendo de esta manera fortalecer mi bolsillo. Además, deseo que genere un aura próspera donde se generen las condiciones óptimas para aportar al país mi conocimiento en programación de software. Que Dios y la Patria lo bendigan Javier, pero, por sobre todo, Viva La Libertad Carajo. Firma: Mateo Beconi.  Máximo Morassutti (IG: @politicomorassutti) – 14 años – Provincia de Mendoza “Recuerdo que los cuerpos de intelectuales hablaban de perderlo todo. Hablaban de “democracia” en un país donde reinan censuras y amenazas, de “soberanía” con una moneda sin valor, de “gratuidad y calidad” con hospitales sin insumos, de “derechos” con escuelas públicas deficientes donde no existe incentivo a la mejora y predominan docentes que lejos de educar, adoctrinan. De “dignidad” con jubilados pobres, y de “patria”, en un contexto donde jóvenes, a pesar de poseer todas las capacidades, se ven privados del mundo laboral, limitando el sueño de la casa propia a una notebook. Sin olvidar aquellos que trágicamente ni siquiera llegaron a vislumbrar ese sueño, porque su vida les fue arrebatada en un pestañeo por un motochorro. Hoy en día los argentinos necesitan un fuerte trabajo en dos ámbitos de urgencia: una justicia que verdaderamente se cumpla con la premisa clara: “quien las hace, las paga”, y una economía que permita prever un proyecto de vida: donde los políticos no puedan tapar sus desastres recurriendo a la emisión monetaria, donde podamos estar seguros de que ahorramos en una moneda fuerte, donde no se ceda al gasto entendiendo que los recursos son escasos para necesidades infinitas, donde se permita el libre comercio y se dé lugar a la competencia…” Estimado Presidente Electo Javier Milei. En sus manos reposa la esperanza de la población agotada de trabajar sin poder permitirse la planificación, del productor condenado a los impuestos, de las familias víctimas de la delincuencia como las de Cecilia y de Lucio, de la cara golpeada de la militante Maribel, del jubilado Mario… todos ellos, los argentinos de bien, sintieron romper sus cadenas cuando se anunció tu victoria. La tarea por delante es ardua, pero sé que usted cuenta con la convicción necesaria para no desistir en ninguna de sus propuestas. Quienes entendemos que este país debe cambiar, te proporcionaremos sin dudar la paciencia y el acompañamiento necesario para superar los desafíos, y de esta manera, construir una Argentina libre que resplandezca en todos los aspectos, como lo conseguimos con las ideas alberdianas. Así que adelante, que no te ciegue el miedo de nadie, porque el camino hacia el progreso sólo comienza con pasos firmes. Atentamente, Máximo Morassutti. Aprendiz de Economía. Articulista de críticas y reflexiones políticas.  Nicolás Schell  (IG: schell.nicolas) – 23 años – Provincia de Corrientes Sr. Presidente de la Nación, Javier Gerardo Milei.  Estimado, un placer saludarlo, Soy Nicolás Antonio Schell, presidente de la agrupación “Activistas por la Libertad”, le envío esta carta de agradecimiento por tomarse el tiempo de leernos, así mismo lo felicito por su gloriosa victoria en esta última elección. Usted unió a los argentinos a tal punto que cada uno aportamos en esta campaña un granito de arena porque creemos firmemente en las propuestas liberales. Nosotros los jóvenes esperamos que su gobierno sea exitoso, aunque entendemos las dificultades actuales. Pero estamos aquí para defender las ideas de la libertad como ya lo venimos haciendo desde el 2019, cuando fundamos después de tanto tiempo en la Facultad de Derecho y Ciencias Políticas y Sociales (UNNE) la “Agrupación Liberal Alberdi”. Cómo Correntino, tierra del General San Martín, le pedimos que tenga presente junto con el Gobierno Provincial la realización de acuerdos para que la industria privada creadora del empleo genuino siga en crecimiento, para que el campo sea más libre con menos trabas impositivas, para poder aprovechar al máximo la industria turística en el Iberá y toda la localidad, ya que en región no conocemos el futuro ni progreso porque por malos gobiernos nuestro distrito quedó estancado. Los jóvenes así, en su más mínima posibilidad, abandonan su tierra. Debemos trabajar arduamente para que la libertad llegue por estos lares, para así reconstruir a la región del NEA que tan abandonada se encuentra. Nuestra provincia, siempre discriminada por gobiernos nacionales, hace que le pidamos con fuerza que no se olvide de nosotros, porque cuando la patria está en peligro no se preocupe Sr presidente, que Corrientes le va ayudar. Me despido humildemente de usted. Atentamente, Nicolás.  Pablo Cabrera (IG: @cabrerapabloalexis) – 24 años – Provincia de Entre Ríos Estimado Presidente Electo Javier Milei y Vicepresidente Electa Victoria Villarruel. En nombre de la organización Libertad Avanza Entre Ríos, quiero expresarles nuestro más profundo agradecimiento y admiración por representarnos y liderar un cambio verdadero en nuestro querido país. Su valiente y auténtica campaña política ha despertado un sentimiento de esperanza y renovación en cada rincón de Argentina. Es imposible encontrar palabras suficientes para transmitir la emoción que sentimos al ver plasmado en las urnas el resultado de su incansable trabajo. Lucharon contra vientos adversos, difamaciones y mentiras propias de la vieja política, pero siempre mantuvieron su enfoque en la verdad y en el bienestar de todos los argentinos. Recuerdo con claridad los momentos en los que, como militantes de la libertad, recorríamos las calles más concurridas de la ciudad, repartiendo nuestras boletas, y enfrentando tanto la indiferencia de algunos como el rechazo agresivo de otros. Esos momentos de injusticia y confrontación solo fortalecieron nuestra convicción de que juntos podríamos construir una nación en la que todos pudiéramos sentirnos orgullosos: “Volver a ser potencia mundial” era el grito que entonábamos en cada esquina, convencidos de que era posible alcanzarlo. Es notorio y encomiable el genuino esfuerzo y compromiso que tanto usted, Presidente Electo Milei, como usted, Vicepresidenta Electa Villarruel, han demostrado a lo largo de esta campaña. Aunque carecían de experiencia política, sobraba en ustedes la valentía, la entrega y las ganas de lograr un cambio profundo en nuestra querida nación. Agradecemos especialmente a todas las personas que confiaron en nuestras propuestas y no se dejaron llevar por la campaña del miedo. Fueron ustedes quienes votaron por una Argentina mejor, una Argentina en la que todos tengamos las mismas oportunidades y en la que la libertad sea el motor de nuestro progreso. Nuestro agradecimiento también se extiende a todos los militantes de la libertad que trabajaron arduamente, día y noche, para hacer posible este resultado histórico. Hoy, con la victoria en nuestras manos, nos enfrentamos a una nueva y emocionante etapa: la reconstrucción de nuestro país. Sabemos que el camino no será fácil, pero estamos seguros de que, con su liderazgo y el compromiso de todos los argentinos, lograremos recuperar el esplendor de nuestro suelo, el cual supo ser el más próspero del nuevo mundo. En cada uno de nosotros está la responsabilidad de trabajar por un futuro brillante y justo para todos. Continuaremos junto a ustedes, apoyando las medidas y políticas que promuevan la libertad, el desarrollo económico y la igualdad de oportunidades. Una vez más, les expresamos nuestro más sincero agradecimiento por representarnos y ser la voz de aquellos que anhelan un cambio real en nuestra Argentina. Estamos convencidos de que su liderazgo marcará una diferencia trascendental en nuestro país. Con el más profundo respeto y admiración, Atte: Cabrera Pablo, Libertad Avanza Entre Ríos.  Patricio Aguilera Roque (X: @Mr_Desfase) – 19 años – Provincia de Catamarca Querido Javier Milei, Permítame expresar mi más sincero y emocionado agradecimiento por su reciente elección como presidente de Argentina. Tu victoria ha traído consigo un torrente de esperanza y entusiasmo para muchos de nosotros que anhelamos un cambio significativo en Argentina. Tu intelecto, valentía y pasión inspiran a muchos, incluyéndome. Es con gran entusiasmo que te brindo mi apoyo y confianza en tu capacidad para dirigir nuestro país hacia un futuro próspero y recuperar la potencia que una vez fuimos. Con gratitud y esperanza, Roque Patricio Aguilera Santiago Vieites (IG: @santiago_vieites2605) – 18 años – Provincia de Buenos Aires “Milei representa el hartazgo y a su vez la esperanza de una sociedad atrofiada por el populismo político. Lo veo a Javier como el inicio de un camino, el líder de una verdadera revolución. Nuestro país ha sido víctima de muchos gobiernos los cuales solo han buscado el poder con el fin de enriquecerse a sí mismos, han dejado gente en la pobreza, familias sin hogar, y utilizaron esas figuras como símbolos políticos promoviendo la grieta entre todos nosotros.  Lo veo como la persona que demostró que la grieta no es entre ciudadanos, sino entre los políticos que han malversado la administración pública y los ciudadanos de este país que han trabajado insaciablemente solo para pagarle todos los lujos al Estado.  Me gustaría ver, en estos 4 años por lo menos, el inicio de un cambio, las primeras luces verdes de nuestro camino. Quisiera que muchos de los delincuentes, a los que Javier correctamente llama casta, paguen por las atrocidades que le hicieron a la patria. Quisiera ver un país donde los jóvenes empiecen a replantearse la idea de irse del país, y decidan, justificadamente, quedarse. Quisiera ver un país donde se cumpla y se respete la ley. Quisiera ver un país más floreciente, donde no se trabaje como esclavo para entregarle el fruto de tu trabajo al gobierno, sino, uno donde los argentinos gocen de libertad y prosperidad en sus vidas. Estas elecciones fueron claras, a pesar de que el resultado de las elecciones generales fue sorprendente, el balotaje lo dejó todo bastante claro. En ningún momento de la campaña dejé de respirar aires de cambio y libertad. Los argentinos eligieron a la persona que creen más capaz de liderar este cambio, tanto fue así que Javier Milei se convirtió en el presidente más votado en la historia de nuestra democracia moderna.  Santiago Benítez (X: @Santitoo_03) – 20 años – Provincia de Formosa Estimado señor presidente Me dirijo a usted en calidad de ciudadano residente de la provincia de Formosa con el agrado de primero felicitarlo por ser el nuevo presidente electo de la República Argentina, cuenta con todo mi apoyo y el de la mayoría del pueblo argentino, juntos le ayudaremos a que este país ocupe el lugar en el mundo que nunca debió dejar, defendiendo la competencia, la libertad y la propiedad privada.  Muchas gracias por su atención. Atentamente, Benitez Santiago Emanuel Epílogo – Por Jeremías Rucci (X: @jeremias_rucci) – 21 años Queridos Máximo Morasutti y Lionel Schroeder; Estimado Javier, Es un honor para mí expresar mi agradecimiento por la amable invitación a poner el epílogo a esta carta en respaldo al presidente electo liberal libertario, Javier Milei. Los más de 23 jóvenes que colaboramos en esta carta lo hicimos impulsados por la esperanza de ver la libertad como el catalizador del cambio para nuestra amada nación. Javier Milei encarna la esperanza de millones de argentinos que lograron derribar el muro del kirchnerismo, que durante más de 20 años despojó a nuestra nación de sueños y esperanzas. Por eso, apreciado Javier, a partir del 10 de diciembre de 2023, con tu liderazgo y el respaldo de los argentinos, comienza un nuevo capítulo para Argentina. Una etapa llena de vigor, repleta de esperanza y libertad. Este nuevo amanecer es un llamado a la acción, una invitación a explorar horizontes de progreso y bienestar a través de los principios fundamentales del liberalismo. La travesía que emprenderás como líder de nuestra nación no estará exenta de desafíos. Enfrentarás críticas, resistencia y obstáculos, pero confiamos en que tu determinación y visión llevarán a Argentina hacia un futuro más brillante. La responsabilidad que asumes es monumental, pero también es una oportunidad única para redefinir el rumbo de nuestro país, devolviendo el poder a los ciudadanos y construyendo un legado de libertad y prosperidad para las generaciones venideras. Este momento crucial exige que enfrentemos los desafíos con valentía y determinación. La historia nos ha enseñado que los mejores momentos de nuestra amada Nación fueron con Libertad, donde las botas del estado estaban lejos de los ciudadanos. En esta nueva etapa, te instamos a mantener la llama de la libertad ardiendo, a defenderla contra vientos contrarios y a trabajar incansablemente por un país donde cada argentino pueda perseguir sus sueños sin miedo a la opresión. Que la libertad sea el hilo conductor que te guíe en la toma de decisiones, recordándonos a todos que la grandeza de una nación radica en la libertad de sus ciudadanos. La oportunidad de forjar un futuro mejor está frente a nosotros, Javier. El momento es ahora o nunca.  Lapidario informe del Massazo: Los ultimos numeros muestran que Massa dejó un salario mínimo de 152 dólares en Argentina Gracias al DNU de Milei, las empresas están dejando de retener obligatoriamente la cuota sindical del sueldo de sus empleados Empezó la Motosierra en la TV Pública: Se eliminaron dos noticieros, móviles a la costa y toda la grilla militante Con solo 20 días en el cargo y la Ciudad en emergencia, Jorge Macri se fue de vacaciones a Punta del Este en jet privado Estallido social en La Rioja de Quintela: La policía y los empleados públicos denuncian salarios impagos Gracias a las medidas de Milei, se abren nuevos mercados en Brasil y China para la exportación de productos argentinos Periodista que expuso el Pizzagate y produjo un documental en contra de una red global de pedófilos, fue hallada muerta Cavallo acertó la inflación de diciembre con una increíble exactitud, y anticipa que volverá a un dígito mensual para el mes de abril ¡CONTAMOS CON TU AYUDA! Este proyecto, este equipo, y este diario son sólo posibles gracias a la contribución de cientos de donaciones de nuestros lectores todos los días. CONTACTO: Para contactarse con nosotros: contacto@maderomediagroup.com  Copyright © 2023."
result = spanish_lemmatizer_with_pos(text_to_lemmatize)

# Print the result
for token_info in result:
    print(f"Token: {token_info[0]}, Lemma: {token_info[1]}, POS: {token_info[2]}")



Token: Argentina, Lemma: Argentina, POS: PROPN
Token: y, Lemma: y, POS: CCONJ
Token: un, Lemma: uno, POS: DET
Token: giro, Lemma: giro, POS: NOUN
Token: transcendental, Lemma: transcendental, POS: ADJ
Token: en, Lemma: en, POS: ADP
Token: su, Lemma: su, POS: DET
Token: historia, Lemma: historia, POS: NOUN
Token: :, Lemma: :, POS: PUNCT
Token: El, Lemma: el, POS: DET
Token: voto, Lemma: voto, POS: NOUN
Token: a, Lemma: a, POS: ADP
Token: Milei, Lemma: Milei, POS: PROPN
Token: es, Lemma: ser, POS: AUX
Token: la, Lemma: el, POS: DET
Token: búsqueda, Lemma: búsqueda, POS: NOUN
Token: de, Lemma: de, POS: ADP
Token: la, Lemma: el, POS: DET
Token: cura, Lemma: cura, POS: NOUN
Token: para, Lemma: para, POS: ADP
Token: la, Lemma: el, POS: DET
Token: enfermedad, Lemma: enfermedad, POS: NOUN
Token: ,, Lemma: ,, POS: PUNCT
Token: https://derechadiario.com.ar, Lemma: https://derechadiario.com.ar, POS: PROPN
Token: /, Lemma: /, POS: PROPN
Token: opinion, Lemma: opinion, POS: PROPN
Token: /, Lemma: /

In [13]:
import spacy

def spanish_lemmatizer_with_pos(text):
    # Load spaCy model for Spanish
    nlp = spacy.load('es_core_news_sm')

    # Process the text using spaCy
    doc = nlp(text)

    # Extract lemmatized tokens with POS information
    lemmatized_text = ' '.join([token.lemma_ for token in doc])

    return lemmatized_text

# Example usage
text_to_lemmatize = "Argentina y un giro transcendental en su historia: El voto a Milei es la búsqueda de la cura para la enfermedad,https://derechadiario.com.ar/opinion/argentina-y-un-giro-transcendental-en-su-historia-el-voto-a-milei-es-la-busqueda-de-la-cura-para-la-enfermedad,laderechadiario,right_wing,Publicado en Por El diputado nacional fue el candidato más votado en las primarias y agradeció “a todos los que vienen apostando desde el 2021 en crear un proyecto liberal, con proyección nacional y que llegue a ser gobierno”.   Los comicios del pasado domingo 13 de agosto en Argentina no pasarán desapercibidos en su historia. Sucede que el candidato libertario Javier Milei dio el batacazo imponiéndose en las urnas por encima de las dos fuerzas políticas hegemónicas, Juntos por el Cambio, respaldado por la UCR, y Unión por la Patria, respaldado por el PJ.  Si bien es cierto que esto no es algo definitivo dado que se trata de elecciones primarias y la distancia entre los tres primeros candidatos no es significativa, no deja de representar un hito histórico en una sociedad víctima de una serie de mitos y construcciones sociales que afectaron severamente su moral. Argentina es un país que ha naufragado por terribles vaivenes políticos, institucionales y económicos a lo largo de su historia reciente: golpes de estado, dictaduras, dos hiperinflaciones, una guerra, crisis económicas recurrentes, etc. Estos antecedentes no son menores porque son los encargados de enmarcar los valores morales de una sociedad.  Históricamente los argentinos fueron víctima del saqueo institucionalizado donde los gobernantes los han engañado de forma sistemática y permanente. Entre los tantos ejemplos que vienen a mi mente lo constituye el Banco Central y varios impuestos que fueron creados de “forma transitoria” con la promesa de ser “eliminados” en el “corto plazo” para convertirse en definitivos conviviendo con la gente incluso hasta nuestra actualidad. Se les confiscaron a las personas sus ahorros sin piedad alguna, la pobreza y la inseguridad se volvieron moneda corriente y la libertad se convirtió en una expresión vacía de contenido. Argentina es un país donde una gran cantidad de gente no cree en la propiedad privada, pilar fundamental de la civilización. Esta última es solo una “cuestión relativa”, algo que puede prostituirse en función a las “necesidades políticas” del momento, para muchas personas. Es por ello que el argentino promedio está enfermo, ya que está infectado de mentiras y discursos que le hicieron creer que todos los males son culpa de los “inescrupulosos y codiciosos empresarios” que solo quieren “cagar a la gente”. Mientras que en realidad el Estado (ellos, los burócratas estatales) son el mesías al cual deben confiarle todo: la salud, la educación, la seguridad, la vivienda, etc.  A lo largo de generaciones se habló de esto, de la “justicia social”, caracterizada por un fuerte y marcado paternalismo estatal. La justicia social, por más “bonito” que pueda sonar en términos semánticos, no es justicia ni es social.  Esto es porque consiste en un trato desigual ante la ley, castigando la propiedad privada de quienes mejor se desempeñan en el proceso productivo para financiar la maquinaria de “derechos” convirtiendo en rehenes a quienes son esquilmados a impuestos y a quienes son “beneficiarios de los derechos” generándose así un círculo vicioso de dependencia estatal permanente.  Si hacemos un análisis profundo y racional del mensaje que se oculta detrás de la justicia social podremos concluir que es el siguiente: “como vos nunca serás capaz de pagarte la salud, la educación, la seguridad y la vivienda por vos mismo, nosotros, representantes iluminados del Estado, ente supremo y superior a vos, haremos que puedas acceder. Me encargare de dirigir tu vida por siempre porque tu incapacidad no te permitirá progresar, necesitaras siempre de nosotros”. Desde nuestros abuelos este adoctrinamiento en la religión del estado se hizo institucional “encarnandose” en el ideario popular. Hace casi un siglo que las personas no confían en sí mismas. El individuo fue degradado a la mínima expresión.  La sociedad decidió delegar sus propias responsabilidades individuales en el colectivo con la creencia falaz de que los gobernantes se encargarían de trabajar en pos del “bienestar general”. Es por lo que, con toda lógica, hablar hoy de “vouchers educativos” o “seguro de salud universal” o de “libre mercado” suena como una broma de mal gusto o “mala palabra”. Es a partir de esta lógica que la clase política lucró, lucra y seguirá lucrando hasta tanto sepamos reaccionar. Estamos ante verdadero despertar de la sociedad, donde Milei será solo un mero protagonista pasajero. Por ello, será importante que comprendamos que los verdaderos cambios no vendrán a través de Milei (ni de ningún político) en democracia, sino más bien a través de nosotros mismos, los verdaderos protagonistas, los individuos. Existe hoy en la sociedad un fenómeno disruptivo de individuos que comienzan a creer en ellos mismos, en su libertad, en su razón, en sus capacidades y que, con el tiempo, nada ni nadie les podrá arrebatar su libertad bajo ningún tipo de engaño colectivista. ¡Despertad, aún no hemos roto nuestras cadenas! Por Hernán Vionnet, para La Derecha Diario.  No, no son lo mismo: Argentina tiene la oportunidad de elegir a los mismos de siempre o apostar por algo distinto Por qué Milei: El candidato de la Argentina postergada se abrió camino en las urnas Milei en Davos: En un discurso histórico, defendió las ideas libertarias, apuntó contra la justicia social y la Agenda 2030 Fuerte recomposición de Reservas del BCRA: Se logró comprar más de US$ 4.600 millones desde el 11 de diciembre El Gobierno de Milei lanzó un operativo contra los pesqueros chinos que depredan ilegalmente al Mar argentino Milei le ahorra al Estado argentino casi 400.000 dólares viajando a Davos en un vuelo comercial con escala en Frankfurt Los fabricantes de maquinaria agrícola apuestan a la reactivación de la economía por las reformas del Gobierno de Milei La OMS y el Foro de Davos se preparan para una “Enfermedad X” que podría volver a llevar al mundo a cuarentena Publicado en Por Todas las condiciones están dadas para que el hombre más rico del mundo, que ya está mirando a la Argentina con Starlink, piense en invertir con su otra empresa: Tesla.   El triunfo de Javier Milei, las características de los recursos argentinos y el aspecto socio-económico del país indican que ha llegado una situación muy favorable para que Tesla, la mega-compañía de autos eléctricos de Elon Musk, abra una fábrica (conocidas como Gigafactory) en Argentina. El presente análisis, tiene como objetivo abordar los aspectos más relevantes de esta pregunta, y romper con una serie de mitos sobre el litio, el daño ambiental y la economía argentina. Permítame presentarme: soy Félix, un profesional con cinco años de experiencia en el sector de las Energías Renovables.  He trabajado en proyectos de escala eléctrica en Chile y proyectos de generación distribuida en Argentina. Mi función involucra la gestión de proyectos y la supervisión de asuntos relacionados con los impactos ambientales. Además, soy licenciado en Gestión Ambiental.   Argentina es un país rico en recursos naturales y tiene amplio territorio disponible para nuevos proyectos. Desde el punto de vista eléctrico, particularmente en Energías Renovables, el país posee condiciones óptimas para el desarrollo del sector.  El territorio argentino tiene un potencial significativo para el desarrollo de la Energía Solar, particularmente en las regiones andina y subandina, desde Jujuy hasta Neuquén. La Puna y la Quebrada de Humahuaca también exhiben niveles sustanciales de radiación, con una irradiancia horizontal global (GHI), que oscila entre 2400 y 2700 kWh/m2.  Además, Argentina es un participante notable en la energía eólica, ubicándose entre los seis primeros países con patrones eólicos consistentes, con un factor de capacidad promedio anual del 20%. En consecuencia, una potencial Gigafactory en el país podría utilizar y suministrar fácilmente energía limpia a todo el complejo. Además, existe potencial para establecer un Parque Solar para generar energía para el sistema eléctrico argentino.  En el ámbito del desarrollo de baterías eléctricas, el recurso más importante es el litio, del cual Argentina ocupa el segundo puesto con reservas mundiales con 98 millones de toneladas de carbonato de litio (LCE). Aún con una severa subinversión, es el cuarto productor a nivel mundial, después de Australia, Chile y China, con una capacidad de producción anual de 37.500 toneladas. El triángulo del litio -formado por Argentina, Bolivia y Chile- posee alrededor del 65% de los recursos mundiales de litio. Esto no sólo ha atraído la atención de las empresas mundiales de vehículos eléctricos, sino que también ha estimulado las inversiones en Argentina. Ejemplos notables incluyen el Grupo Chery, que planea construir una planta en el país para producir vehículos eléctricos y baterías de litio.  Asimismo, Stellantis, producto de la fusión de FCA (Fiat-Chrysler) y PSA (Peugeot-Citroën), anunció una inversión de 90 millones de dólares para adquirir una participación del 19,9% en la empresa argentina Lithium & Energy (ALE). Este movimiento estratégico tiene como objetivo asegurar un suministro anual de 15.000 toneladas de litio, facilitando su uso en baterías de vehículos eléctricos.  Establecer una Gigafactory cerca de una de las reservas de litio más grandes del mundo podría generar algunos de los costos de producción más bajos para vehículos eléctricos a nivel mundial, especialmente considerando los abundantes recursos solares en la región. Además, la proximidad de Argentina a la Cordillera de los Andes contribuye a tener uno de los sectores mineros más ricos del mundo. Entre los recursos minerales del país destacan nitratos, yodo, sal común, sales de litio y potasio, boratos, áridos, arcillas, calizas, yeso, cuarzo, barita, caolín, feldespato, talco, diatomita, bentonita, dolomita, guano, apatita, zeolitas y perlita. También es fundamental resaltar que el precio del litio alcanza consistentemente máximos históricos año tras año. Así, Tesla no sólo podría establecer su Gigafactory en Argentina, sino también involucrarse en el sector minero para reducir los costos de producción.   Habiendo trabajado personalmente en el desarrollo de proyectos en Chile y Argentina y habiendo discutido permisos ambientales con colegas, puedo dar fe de que obtener la aprobación para la construcción no es tan burocrático como en algunos países vecinos, como en Chile.  La evaluación de impacto ambiental y el desarrollo del proyecto podrían procesarse en un período relativamente corto para iniciar el proyecto. Se requerirán estudios ambientales con consultores locales según sea necesario tanto para el proyecto minero como para la Gigafactory, pero la perspectiva es muy positiva. En conclusión, creo que Argentina está estratégicamente posicionada por su cercanía a recursos, lo que la convierte en uno de los países donde los costos de construcción de vehículos eléctricos son más económicos de la región. Los recursos solares podrían generar energía para la minería, la Gigafactory, y el excedente de energía podría venderse a la red.  La proximidad de la Gigafactory a su fuente de litio podría reducir significativamente los costos de litio de Tesla para la producción de vehículos, mejorando la eficiencia en comparación con la competencia. Además, la disponibilidad de diversos recursos en la región andina y la facilidad para obtener permisos de construcción hacen que Argentina sea ambientalmente ideal para tales emprendimientos.  Históricamente, Argentina fue uno de los países más innovadores de la región durante tiempos de mayor libertad económica. A continuación se detallan algunos de los inventos más importantes creados por argentinos, que demuestran la capacidad y talento para el trabajo en nuestro país: 1. Amortiguador hidroneumático (Francisco Abolio, 1930). 2. Bolígrafo. 3. Bypass coronario de René Favaloro. 4. Sistema de autobuses públicos (Colectivo). 5. Corazón artificial (Domingo Liotta y Denton Cooley, 1969). 6. Método Galli Mainini para el diagnóstico precoz del embarazo. 7. Dibujos animados. 8. Encendedor piezoeléctrico Magiclick de Hugo Kogan (1969). 9. Helicóptero de Pescara. 10. Identificación dactilar de Vucetich. 11. Instrumentos médicos de Favaloro, Finochietto y otros. 12. Instrumento de navegación nocturna para aeronaves (Almonacid, 1925). 13. Jeringa desechable (Carlos Arcuzzi, 1979). 14. Marcapasos cardíaco (Francisco de Pedro, 1979). 15. Membrana de colágeno curativa (Celia Mohadeb, 1999). 16. Método de conservación de sangre con citrato de sodio por Agote. 17. Motor de combustión interna “Péndulo” (Eduardo Taurozzi, 1970). El talento argentino en innovación y persistencia siempre se ha destacado en el tiempo. Lamentablemente, los problemas económicos y las excesivas regulaciones han reducido la capacidad de innovación, pero las capacidades de los argentinos siguen siendo evidentes, demostrando dedicación en todos sus emprendimientos.  A medida que Argentina comience a crecer económicamente, podemos esperar ver mentes brillantes brillar una vez más en nuestro país. A pesar de soportar años de desafíos económicos, muchos argentinos han brillado tanto en el país como globalmente por sus capacidades, y muchos otros aún tienen que demostrar su potencial.   Pero lo más importante, el desarrollo de la Gigafactory generará oportunidades de empleo de alta calidad en la zona, ya que será necesario emplear una gran parte del personal utilizando mano de obra local, además de ingenieros, geólogos, y expertos que conozcan la zona. Esto también tendrá un impacto positivo en los servicios de la zona, como alojamiento, sanidad, luz, ya que deberán ser desarrollados como apoyo a la mega-fábrica. Respecto a la locación, la región norte del país cuenta con extensos terrenos, minimizando la posibilidad de impactar negativamente las actividades sociales. Las comunidades indígenas más conflictivas se encuentran en en el Sur del país. Optar por el desarrollo en la región Norte, donde la Gigafactory y las actividades mineras se consideran óptimas, minimiza la probabilidad de encontrar problemas con las comunidades locales. Debajo se adjunta una imagen que muestra la distribución geográfica de las comunidades indígenas en Argentina, para dar sustento a los puntos previamente mencionados:  Sin duda, uno de los factores más cruciales a destacar es la situación económica de Argentina. El país ha experimentado intervenciones gubernamentales prolongadas, interfiriendo con el sector privado y provocando un empobrecimiento cada vez mayor debido a la falta de inversión y generación de riqueza. Las medidas de justicia social, destinadas a redistribuir la riqueza de los productores a los pobres, han demostrado conseguir el efecto inverso en Argentina, lo que ha resultado en que el 50% de la población viva en la pobreza, con 6 de cada 10 niños enfrentando inseguridad alimentaria. Estas políticas sociales no sólo no han logrado beneficiar a la sociedad como se esperaba, sino que también han impactado negativamente al sector privado. Hace pocas semanas, Javier Milei, el primer economista libertario en asumir la Presidencia, señaló un posible cambio en el futuro de Argentina con reformas liberales.  Aquí hay una breve descripción de las reformas económicas propuestas por Milei 1. Reforma monetaria: La adopción del dólar estadounidense como moneda oficial significa un cambio monumental en el panorama económico de Argentina. Al vincular la moneda a un estándar reconocido mundialmente, Milei pretende infundir confianza a los inversores, mitigar las preocupaciones sobre las fluctuaciones monetarias y fomentar un entorno financiero más estable. 2. Reducción de la intervención gubernamental: La propuesta de Milei de minimizar las funciones del Estado se alinea con los principios económicos liberales. Esto incluye simplificar los procesos regulatorios, reducir los obstáculos burocráticos y crear una atmósfera más favorable a los negocios para las corporaciones extranjeras. 3. Impacto potencial en las inversiones extranjeras: Si se implementan efectivamente, estas políticas económicas propuestas pueden atraer a inversionistas extranjeros que buscan estabilidad y oportunidades de crecimiento. El énfasis en soluciones impulsadas por el mercado y la reducción de la interferencia gubernamental podrían posicionar a Argentina como un destino atractivo para empresas, como Tesla, que buscan expandir sus operaciones.  4. Desregulación de la economía: A solo una semana de la presidencia de Milei, se están realizando esfuerzos para implementar importantes reformas de desregulación económica. Se han eliminado numerosos ministerios redundantes y se han derogado más de 300 leyes que perjudicaban a la economía. Si bien invertir en Argentina todavía puede plantear riesgos, con el ambicioso plan de Javier Milei y el rápido inicio de reformas, podríamos estar ante el surgimiento de una nueva potencia económica en la región. Ser uno de los primeros socios del gobierno en un período tan transformador podría ser una medida estratégicamente inteligente, que podría generar beneficios y oportunidades sustanciales para quienes participan.  La sinergia potencial de factores sociales, ambientales y económicos en Argentina crea una narrativa convincente para que Tesla explore el establecimiento de una Gigafactory y una operación minera en el Norte argentino.  Si bien los desafíos persisten, la perspectiva de alinearse con un panorama político y económico cambiante, aprovechar recursos abundantes y contribuir al desarrollo sostenible posiciona a Argentina como una frontera prometedora para la expansión estratégica de Tesla.  Publicado en Por El fallido operativo de la Policía de la Ciudad para acatar el Protocolo Antipiquetes en la marcha de este 20 de diciembre vuelve a poner en tela de juicio la necesidad de que la Policía Federal intervenga en la dispersión de manifestantes que atacan el microcentro porteño.   En ocasión de la primera marcha piquetera de protesta contra el gobierno del Presidente Javier Milei, resulta de mucha utilidad repasar algunos conceptos fundantes que hacen a la siempre conflictuada noción de “orden público” en el territorio de la Ciudad Autónoma de Buenos Aires, Capital de la República Argentina. No es sorpresivo que el accionar piquetero tienda a concentrarse en la Capital. Después de todo, el objetivo final de las organizaciones que marchan es condicionar al poder político a fin de obtener privilegios y beneficios para las supuestas situaciones de vulnerabilidad que revistan. Párrafo aparte merecería el destino de los fondos que mensualmente perciben de parte de los organismos del Estado Nacional, pero dejando de lado ese detalle es que nos remitimos a la cotidianidad de los trabajadores que deben transitar por las zonas afectadas por las marchas piqueteras. En la Ciudad de Buenos Aires circulan más de diez millones de personas por día por motivos laborales. Gran parte de esta cifra, se ve obligada a transitar por el llamado “microcentro”, zona geográfica predilecta para el desarrollo de actividades económicas, y por eso también, la favorita para los piqueteros.  Durante más de veinte años la sociedad se ha acostumbrado al sometimiento que las organizaciones sociales ejercen sobre el pueblo trabajador: desde cambiar el recorrido diario, llegar más tarde al trabajo o inclusive la suspensión de la jornada laboral en días de marcha, son solo algunas de las cientos de consecuencias que las movilizaciones producen.  Sin embargo, y pese a la ilegalidad manifiesta del accionar piquetero, el poder político no había abordado la tortuosa temática por un mismo motivo que fue denominador común sin importar el color partidario del gobernante de turno: la corrección política. Este miércoles, finalmente, el flamante Presidente Milei decidió que era hora de cortar con los intentos extorsivos. En el marco de un operativo de seguridad sin precedentes, la Gendarmería Nacional, Policía Federal, Prefectura Naval, Policía de Seguridad Aeroportuaria y el Servicio Penitenciario Federal blindaron las calles aledañas a Plaza de Mayo, no sin previamente cerciorarse de disponer delegaciones tácticas en estaciones de tren, accesos automotores, peajes y en localidades del Gran Buenos Aires donde se desactivaron células piqueteras durante toda la noche.  El operativo de seguridad tenía como objeto impedir que los piqueteros cortasen la circulación en las principales arterias de la Ciudad de Buenos Aires y por ende permitir a todos los ciudadanos ejercer su derecho a la libertad de movimiento. Sí bien los lineamientos generales del plan fueron ampliamente cumplidos generando un “antes y un después” en la manera que el Gobierno Federal aborda la temática de los cortes piqueteros, corresponde hacer una desafortunada observación: el limitado accionar de la policía de la Ciudad de Buenos Aires.  Con el afán de estar a la altura de las circunstancias, el Gobierno de la Ciudad de Buenos Aires (GCBA) ordenó un operativo paralelo presuntamente coordinado con el Comando Unificado de las Fuerzas Federales.  El Centro de Monitoreo Urbano (CMU) de la Policía de la Ciudad fue el escenario donde montó su Cuartel General el Ministro de Seguridad porteño Waldo Wolff quien, desde tempranas horas de la mañana se encontraba en comunicación con Patricia Bullrich.  Durante los momentos previos al apogeo de la marcha alrededor de las 16:00 todo parecía marchar coordinadamente: las Fuerzas Federales y la Policía de la Ciudad conducían ordenadamente a los manifestantes hacia Plaza de Mayo, mayoritariamente por la vereda. Empero, al aumentar el grosor de las columnas, los manifestantes comenzaron a tensionar la situación. En primera instancia, actuó la Policía de la Ciudad en Diagonal Sur mientras su Jefe y Viceministro de Seguridad porteño, Diego Kravetz (quien se encontraba en el teatro de operaciones), alegaba que “no necesitaban del apoyo de las Fuerzas Federales”.  Tal fue así que cuando los manifestantes decidieron avanzar sobre la Policía capitalina, no encontraron mayor resistencia y las prolijas líneas de efectivos fueron fácilmente rebasadas con empujones e insultos mediante. Por este motivo, se ordenó a la Policía Federal hacerse cargo luego de que las columnas -que ya copaban el ancho completo de la Diagonal- pasaron la estatua ecuestre del General Julio A. Roca ubicada en la intersección con la calle Perú. Una vez restablecido el orden y conducidas las poco nutridas columnas manifestantes a Plaza de Mayo por parte de las Fuerzas Federales, los medios de comunicación pudieron averiguar que el Viceministro Kravetz había dialogado con el líder piquetero Eduardo Belliboni, autorizándolo a avanzar sobre la totalidad del ancho de las calles, produciendo indefectiblemente el corte total.  Más allá del intento de connivencia de Kravetz con Belliboni -lo cual coloca en una muy poco confortable situación al gobierno de Jorge Macri- corresponde remitir a la cuestión neurálgica de estos problemas, que es la superposición de facultades entre las Fuerzas Federales y la Policía de la Ciudad. En 1995, un año después del otorgamiento de autonomía a la Ciudad de Buenos Aires por parte de la reformada Constitución Nacional de 1994, el Congreso de la Nación sancionaba la popularmente denominada “Ley Cafiero”, o “Ley que garantiza los intereses del Estado Nacional en la Ciudad de Buenos Aires”.  Dicha pieza normativa impedía al Gobierno de la Ciudad de Buenos Aires a tener una policía propia puesto que disponía que la seguridad armada fuera garantizada por la Policía Federal en el territorio de CABA mientras esta sea la Capital de la República. Tras largos debates y pujas de poder, el Congreso modificó la Ley Cafiero en 2007, mediante la Ley 26.288 para permitirle a CABA montar su propio cuerpo policial y establecer el marco de coexistencia con la Policía Federal. Con la fundación de la Policía de la Ciudad en 2017 luego del traspaso de la Superintendencia de Seguridad Metropolitana de la PFA, CABA obtuvo finalmente su tan deseado brazo armado. Sin embargo, la evidencia empírica arroja que en cuanto al mantenimiento del orden público se refiere, la Policía de la Ciudad tiene importantes oportunidades de mejoras respecto de lo que fue la gestión de la PFA.  El problema principal radica en que se trata de una policía heterogénea, dado que sus miembros provienen de diferentes fuerzas y nunca se logró el aseguramiento de un criterio único de tropa y formación. En los tiempos que se avecinan, donde la resistencia anti-democrática al cambio que necesita nuestro país puede volverse cada vez más violenta, es indispensable contar con un criterio unificado a fines de hacer frente a las amenazas y asegurar la vigencia plena del orden público y del Estado de Derecho.  La Ciudad de Buenos Aires debe estar a la altura de las circunstancias o en su defecto, quizás la alternativa sea volver a modificar la Ley Cafiero a fines de aclarar que las operaciones de establecimiento del orden sean únicamente desarrolladas por el Estado Nacional y sus Fuerzas Federales siempre que el Gobierno Nacional sea el foco de la concentración. Sea del modo que fuere, la Argentina entró en un cambio de época irreversible. Seremos testigos de cómo cada responsable institucional se adapta a las circunstancias.  Publicado en Por Dos estudiantes han tomado la iniciativa de juntar distintos comentarios que jóvenes de todo el país le escribieron al presidente electo Javier Milei para compilar una carta histórica que aquí queda presentada.   A los 7 dias del mes de diciembre de 2023 Señor Javier Gerardo Milei. Presidente Electo de la Nación. Desde las tierras de Chaco y Mendoza, Máximo Morassutti y Lionel Schröder, dos estudiantes de 14 y 17 años, hemos tomado la iniciativa de escribirle esta carta.  Gracias a la colaboración de muchísimas personas, un grupo diverso de jóvenes de todas las provincias del país se une con entusiasmo para expresar apoyo tras su histórica victoria.  Este triunfo representa para nosotros la ruptura de las cadenas que vienen limitando nuestro potencial, con políticas intervencionistas de las cuales. sabemos. Solo se benefician unos pocos. Nos sentimos profundamente identificados con sus ideas y creemos que, bajo su liderazgo, todas las regiones de nuestra nación podrán florecer en un ambiente de libertad, justicia y prosperidad compartida. “Oíd el ruido de rotas cadenas”: un eco del himno que resuena en todos los rincones. López y Planes escribió en 1812 aquel fragmento posiblemente seguro de hacerlo, pero no consciente de que perduraría de manera tan icónica hasta los tiempos que corren… Hoy, consideramos momento de renombrarlo con su elección, entendiendo que nos encontramos frente a la apertura de un nuevo capítulo en nuestra historia: la era liberal.  A continuación, procederemos a presentar llenos de emoción los mensajes enviados por cada uno de los voluntarios: Abi Franzutti (IG: @abifranzutti) – 17 años – Provincia de Córdoba Sr. Presidente Javier Milei. Me complace presentar a usted las más cordiales felicitaciones en nombre de la provincia de Córdoba y en el mío propio, con motivo de su elección como próximo presidente de la República de Argentina. En esta propicia ocasión nos acercamos para formular nuestros mejores deseos de que la gestión de gobierno que usted habrá de encabezar en este hermoso país, haga importantes contribuciones al fortalecimiento de la democracia, al mantenimiento de la paz y la solidaridad en el pueblo argentino, y a un mayor desarrollo económico que propenda al bienestar general. Agregando a la juventud de LLA, le agradecemos por su apoyo que nos brindó en campañas electorales, y nuevamente agradecerle por todo su arduo esfuerzo, trabajo y desempeño para llegar a dónde hoy usted, sr. Presidente, está. Asimismo, Córdoba confía en sus propuestas y esperamos que fielmente así se impulsen en todo nuestro territorio. Con reiteración de nuestras congratulaciones le saluda, atentamente, Abi Franzutti.  Benjamin Gosen (X: @GOSEN_LIBERAL) – 16 años – Provincia de Tucumán Estimado Presidente Electo, Desde San Miguel de Tucumán, brindo todo mi apoyo hacia usted. Desde el interior, confiamos en que pueda poner de pie a Argentina y posicionarnos en el lugar que nos merecemos. Uno de los principales problemas de hoy en día es la inseguridad; por tanto, confiamos plenamente en su compañera de fórmula, Victoria Villarruel, en aquella área. Solo puedo decir a nuestro presidente y vicepresidente electos: ¡Muchas gracias por volver a traernos la esperanza a este hermoso país!. ¡Viva La Libertad! Ezequiel Asenjo (X: @AsenjoEzequiel) – 20 años – Provincia de Chubut Estimado presidente Javier Milei: Quiero expresar mi profundo agradecimiento por su incansable lucha en contra de la corrupción política. Su valentía y compromiso son un ejemplo para todos nosotros. Gracias por defender nuestros derechos y principios con vehemencia.  Sería interesante escuchar las opiniones de Javier Milei sobre los sindicatos y la producción de petróleo en Chubut y la Patagonia, considerando su relevancia en la economía latinoamericana. Sus perspectivas podrían aportar nuevas ideas y enfoques para abordar estos temas.  Gracias Javier milei por devolver la confianza a la gente, espero que le vaya muy bien en su mandato. Facundo Albiñana (X: @Facundo_AlbZ) – 17 años – Provincia de San Juan Estimado Javier Milei: Los jóvenes liberales de San Juan estamos entusiasmados con la vuelta de una Argentina próspera. El camino a seguir implica valentía, paciencia y perseverancia, pero estamos dispuestos a afrontar los desafíos, ya que como decía nuestro prócer Sarmiento, “Si no existieran dificultades, no habría éxitos”. No nos queda más que agradecer el trabajo de tantos años del presidente Javier Milei, en divulgar las ideas de la libertad que hicieron despertar la esperanza en un futuro mejor a la mayoría de los jóvenes de este país.  Gonzalo P. Aguilar Pouilloux (IG: @gonza.agp) – 19 años – Tierra del Fuego UNA ARGENTINA CON ESPERANZAS. Argentina, patria mía, pueblo que pude haber pensado, como tantos otros compañeros, que no podía cambiar, parecíamos destinados al fracaso, pero hace unos días hubo un domingo, ese día histórico donde el liberalismo y la derecha se impusieron ante el avance del totalitarismo, donde el pueblo se puso de pie, y gritando “¡LIBERTAD!” dijo basta a décadas de decadencia; pero esto hay que construirlo, pueblo mío, queda mucho por hacer, mucho por luchar, mucho por ganar, queda confiar en que todo saldrá bien, pero guardo mis grandes esperanzas en que el león seguirá rugiendo. ¿Si tanto hemos avanzando en la batalla cultural por la libertad, porque no podíamos ganarla totalmente? Las fuerzas se han unido en busca de una argentina libre, con orden y donde la gente luche por su vida, y respete la del otro, donde el delincuente pague el ser delincuente, aún si el delincuente es el político. Las derechas del mundo avanzan ante la caída del muro progresista de la censura y la corrección, el pueblo debe mirar a su alrededor, debe mirar a Venezuela, a Cuba, mirar las contradicciones progresistas, mirar su apoyo a delincuentes, terroristas y degradaciones morales, y ver que algo está claro, la lucha es por la moral, la lucha es por la ética, es por la patria, es por las instituciones naturales, medios entre el individuo y el Estado, esas que hay que defender y fortalecer. El hecho es luchar por la libertad, pero esta como medio, porque siempre será un excelente caballo, pero hay que tener hacia donde cabalgar, y eso es lo que venimos a proponer en esta nueva esperanza, un camino, que no pierdas lo que te queda, y que recuperes lo que perdiste, porque si ya te quitaron todo, y te quisieron dejar en la miseria, ahora nos levantamos para decir que la miseria son ellos, y que la libertad no la cederemos. Ahora queda construir la libertad, mis patriotas, sin fanatismos, y sin miedo, porque al totalitario se lo lucha, allí no habrá tolerancia soportable. ¡Viva la libertad!   Guillermo Esteban Romeo (X: @guillernoeromeo) – 20 años – CABA (Buenos Aires) “Es un honor representar en este mensaje las palabras que los jóvenes de CABA tienen para dirigirse a nuestro Presidente Electo Javier Milei.  A lo largo de los años, desde el 2021, hemos experimentado una gran alegría por su participación en la política. Desde su escaño en el Congreso, representando a los ciudadanos, hemos observado en usted la verdadera vocación de servicio a la patria. Nosotros, los jóvenes, formamos parte de un acontecimiento histórico que marcará un antes y un después. Desde las diversas agrupaciones y grupos presentes en la ciudad, hemos trabajado incansablemente para inculcar en los porteños las ideas que condujeron a que usted sea el Presidente en la actualidad. En mi caso, fue a través de la Agrupación Enlaces, donde dedicamos esfuerzos a la campaña tanto de usted como de Ramiro Marra. Queremos expresar nuestro total agradecimiento por lo que ha realizado por el país y por lo que está llevando a cabo. Le agradecemos por su humildad, coraje y, sobre todo, por haberse enfrentado valientemente a un sistema que estaba en su contra. Desde nuestro compromiso en la militancia, nos comprometemos a continuar librando la batalla cultural que ha resultado victoriosa. Nos comprometemos a defender las ideas de la libertad hasta el final de nuestras vidas, porque como dijo nuestro referente Alberdi, “No hay libertad en un hombre donde su seguridad, su vida y sus bienes están a merced del capricho de un mandatario”. Gracias por liberarnos del estado opresor y por asumir la gran responsabilidad que lleva hoy en día. Enviamos un saludo en representación de todos los jóvenes de la Ciudad Autónoma de Buenos Aires.” Saludos cordiales, Guillermo Esteban Romeo.  Javier Arce (X: @XaviArce20) – 23 años – Provincia de Misiones Estimado Pdte. Javier Milei Mi nombre es Javier Arce, y me dirijo a usted para expresarle mi más sincero apoyo y agradecimiento por su candidatura a Presidente de la Nación. Como usted sabe, Argentina atraviesa una profunda crisis económica y social. Los últimos años han estado marcados por el populismo y la incompetencia, que han llevado al país a la ruina. Usted es la única esperanza que tenemos de salir de esta situación. Sus propuestas, basadas en la libertad económica y el respeto a la propiedad privada, son las únicas que pueden llevar a Argentina al progreso.  Aprecio su valentía y honestidad: eso es lo que necesitamos en un líder. Estoy seguro de que, con su ayuda, Argentina podrá volver a ser un país próspero y libre. Atentamente: Javier Arce  Javier Pérez Coy (X: @Coy_javito) – 20 años – Provincia de La Rioja Estimado Presidente Electo Milei. Le escribo desde Chilecito, La Rioja, para expresarle algunas inquietudes que afectan a mi barrio y a nuestra comunidad. Actualmente, vivimos en condiciones precarias, con sueldos bajos y calles sin asfaltar. Las escuelas públicas carecen de recursos, y la situación no parece mejorar. El intendente Rodrigo Brizuela Idoria, a pesar de sus discursos, no ha llevado a cabo obras significativas en nuestro barrio. Rodrigo Quintela, por otro lado, ha centrado sus esfuerzos en un hospital en construcción desde hace mucho tiempo, pero aún sin completar. En cuanto a la seguridad, aunque no hay mucha criminalidad, la presencia de drogas, a veces adulteradas, es preocupante. Además, la corrupción policial, con la confiscación y subasta de vehículos, es un problema persistente. En este contexto, veo en su liderazgo una oportunidad para el cambio. Su enfoque basado en ideas y voluntarismo político es lo que nuestra provincia necesita. Carolina Moreno y Martín Menén también son figuras prometedoras que se han conectado directamente con la comunidad. Espero que su gestión traiga mejoras tangibles a nuestra provincia y municipios. Confío en que su compromiso marcará la diferencia. Le agradezco su atención y le deseo éxito en su presidencia. Atentamente.  Javier Pérez Coy  Jazmín Venini (IG: @jazmin_venini) – 21 años – Provincia de Río Negro “En este día tan especial, en representación de los jóvenes de la hermosa provincia de Río Negro, desde la pintoresca ciudad de Bariloche, quiero expresar mi más sincero agradecimiento a Javier Milei por liderar este nuevo gobierno que trae consigo un proyecto de transformación y libertad, tan esperado por todos nosotros.  Es un honor para mí, siendo una joven barilochense de 21 años, ver cómo nuestro país avanza hacia un horizonte de transformación donde los jóvenes de Río Negro nos vemos comprometidos con las ideas del liberalismo y nos sentimos representados por su visión de un futuro más libre y próspero.  Además, como mujer, encuentro inspiración en ver a la vicepresidente Victoria Villaruel a su lado, así como a la líder de La Libertad Avanza en Río Negro, Lorena Villaverde. La presencia de ambas nos llena de esperanza y nos demuestra que la participación activa de las mujeres en la política es fundamental para construir una sociedad más justa.  Compartimos su visión de mejorar la economía, impulsar el cumplimiento de la justicia y hacer frente a las usurpaciones de comunidades falsas indígenas en el territorio nacional. Estamos ansiosos por ser parte de un gobierno que defienda la propiedad privada, el libre mercado y un crecimiento hacia la Argentina de la gente de bien que con su esfuerzo pueda salir adelante.  Admiramos su compromiso con la juventud y su deseo de permitirnos tener una voz y voto en la construcción del futuro de nuestro amado país. Creemos que la participación activa de los jóvenes es clave para lograr una sociedad más dinámica y justa, y estamos emocionados por la oportunidad de contribuir a ese cambio positivo.  Finalmente, agradecemos la posibilidad de tener roles fundamentales dentro del gobierno, lo cual nos brinda la oportunidad de ser agentes de cambio en esta nueva etapa de nuestra historia. Estamos listos para trabajar codo a codo con usted y su equipo para construir un futuro lleno de oportunidades para los argentinos.” Con esperanza y compromiso, agradeciendo esta oportunidad, Jazmin Venini. Líder de la agrupación nacional “Leones Sueltos”.  Jeremías Cares (X: @JrsCares) – 18 años – Provincia de Neuquén Señor Javier Gerardo Milei. Me dirijo respetuosamente para agradecer su gran influencia en mí y en muchos jóvenes del país. En 2020, en la pandemia, descubrí la importancia de la libertad gracias a usted. No solo aprendí sobre economía sino también sobre lo mal que le hizo el Kirchnerismo a Argentina.  El día del balotaje, me alegré demasiado. A pesar de tener conflictos con mis pares por defender las ideas de la libertad, prometí que si perdías me iba del país, pero ganaste y hoy me toca quedarme y luchar por el país que tanto amo. Juan Ignacio Climent (IG: @_juanicliment_) – 21 años – Provincia de Santiago del Estero Estimado Presidente Electo Sr. Javier Milei, Es un honor dirigirme a usted en calidad de ciudadano en conjunto con el resto de jóvenes para expresar mis más sinceras felicitaciones por su elección como Presidente de la República Argentina. Este triunfo representa una luz de esperanza para todos los argentinos que queremos un país distinto. Admiro su compromiso por levantar a la Argentina y colocarla en el lugar que merece estar. Después de años de miseria y decadencia política, es momento de que aquello ocurra; y por esto depositamos nuestra confianza en usted, no nos defraude como nos defraudaron sus antecesores.  Asimismo, quiero resaltar la importancia del diálogo y el consenso en nuestro país, que no es susceptible de homogeneizar. La diversidad, las diferencias y la tolerancia son necesarias para el progreso pacífico y democrático de nuestra Nación. Basta de grieta. Finalmente, y con pleno respeto por la responsabilidad que ha asumido, como santiagueño siento necesario pedirle que durante los próximos años de su gestión tenga presente a mi provincia y a su desarrollo económico y social, ya que aquí la libertad siempre se pone en duda y cuestión. Sin más que agregar, le deseo el mayor de los éxitos en su gobierno y le ofrezco mi apoyo desde mi provincia para lograr los cambios positivos que el país necesita. Estoy convencido de que así será. Atentamente, Juan Ignacio Climent  Lionel Schröder (IG: @lioschroder) – 17 años – Provincia del Chaco “Tengo 17 años, siempre viví en crisis, siempre escucho en las noticias lo corruptos que son los políticos, el hambre que pasa la gente, la inflación que nos hace más pobres, ministros negociando con el fondo, causas de corrupción, muertes, robos, argentinos rebuscándosela para salir adelante.  Eso vi TODA mi vida: argentinos rebuscándosela… Gente de bien, gente con valores, gente con cultura, con trabajos, con sueños, con metas, con hijos.  Todos estos argentinos tienen algo que los limita y limitó siempre: un aparato ESPANTOSO, el monopolio de la fuerza, que con excusas tontas e inocentes avanzan contra la propiedad, contra la libertad de los argentinos, este aparato cínico tiene nombre; el ESTADO.  Desde que nacemos escuchamos esta palabra y nos parece normal. Nos parece normal que el político de turno regule precios, las exportaciones e importaciones. Que se meta en cómo se debe educar a los niños, en que debemos pensar, hasta en qué moneda podemos o no comerciar.  A muchas personas les parecía normal que el estado tenga el poder de encerrarte en tu casa sin poder trabajar, estudiar, caminar, visitar amigos, familia, etc. Con la excusa de una enfermedad, nos privaron de toda libertad. Ahí en el medio de ese momento, donde el estado demostraba realmente lo que era capaz de hacer (mantener a personas presas dentro de sus casas), conocí al que es hoy el Presidente de la República Argentina, que con euforia, pasión y efusividad, defendía ideas que no estábamos acostumbrados a escuchar. Poco  a poco este hombre fue despertando una cuestión nueva en mí (y en muchos más). Una pasión, un sentimiento, un nuevo sueño: VER A LA ARGENTINA LIBRE. Ese Lionel de 14­-15 años entendió que tenía que luchar y tenía que ser la voz, para sacar al maldito estado del medio y que la gente abra los ojos, que podamos prosperar y no tener que migrar para progresar. Hoy. 3 años después, estoy más que agradecido con Javier Milei porque es el hombre que sembró las ideas de la libertad en millones de personas, y junto a miles de jóvenes queremos cosechar esa gran siembra para poder vivir libres con paz, y no olvidarnos nunca de que UN HOMBRE y MILES DE JÓVENES hicieron la Argentina grande de nuevo. Yo hoy puedo estar emocionado, pero el Lionel de 14 años se que lo estaría más, Le doy gracias a Dios por su gran amor y por haber iluminado a Javier, le seguiré pidiendo a Dios que le de sabiduría y que sea el instrumento para levantar a la Argentina. Lucas Ovarzo (IG: @loyarzo901) – 22 años – Provincia de Santa Cruz FORJANDO UN FUTURO. “En medio de desafíos y anhelos compartidos, los jóvenes de Santa Cruz encontramos en Javier Milei, una fuente de esperanza para un futuro vibrante. Su enfoque audaz y compromiso con los principios liberales resuenan profundamente en nuestra generación, que anhela un cambio significativo y progresivo. En el corazón de Argentina y Santa Cruz, vislumbramos la posibilidad de un renacimiento con Milei. El horizonte se ilumina con la promesa de un gobierno transparente, eficiente y comprometido con la prosperidad económica. Anhelamos una provincia donde el talento y la creatividad de los jóvenes se liberen de restricciones innecesarias. Visualizamos un país donde la libertad individual sea respetada, la innovación sea aplaudida y cada ciudadano tenga la oportunidad de contribuir al progreso nacional. Los niños, adolescentes, jóvenes y adultos de Santa Cruz y Argentina nos destacamos al elegir a Milei como la voz que llevará nuestras aspiraciones al escenario nacional. Confiamos en que, juntos, construiremos una Argentina donde la esperanza florezca y las oportunidades se multipliquen para cada generación venidera. Esta elección es nuestra apuesta por un mañana lleno de promesas. En Santa Cruz, se le puso fin al Kirchnerismo caracterizado por la corrupción. Este momento marca un hito significativo en la historia de la provincia, señalando el inicio de una nueva fase centrada en la transparencia, la integridad y el avance.”  Una Argentina es imposible con los mismos de siempre. VLLC! Lucas Oyarzo, referente de Pibes Libertarios S.C  Manuel Agustín Galdame (X: @Manuboy28) – 23 años – Provincia de San Luis Estimado Javier Gerardo Milei. Desde San Luis, festejamos su triunfo con esperanza.  El cambio que su elección representa es muy positivo para Argentina. Tu victoria nos llena de optimismo, confiamos en que vas a liderar al país hacia un futuro próspero y brillante, dónde recuperemos el camino y los valores que nunca debimos dejar atrás. Vamos juntos a un ciclo de crecimiento, libertad y prosperidad. El camino no será fácil, pero “no hay noche que no haya sido derrotada por el amanecer.” ¡VIVA LA LIBERTAD! Marco Guerrero (X: @PrimoG_333) – 31 años – Provincia de Salta Sr, Presidente.   Salta está contigo. Somos una provincia que sufre la pobreza, la falta de vivienda, la precariedad sanitaria y el atraso productivo. Queremos ser libres de la opresión estatal, la corrupción y el saqueo.  Confiamos en usted y en nuestra vicepresidenta, que nos representan con valentía. Salta será grande sólo si es liberal, como usted nos enseña.  Marcos Zapata (X: @Markitos8221) – 17 años – Provincia de Jujuy Estimado Javier Milei: Mis mayores apoyos y buenos deseos hacia usted y su vicepresidente Victoria Villarruel. Fue una campaña desafiante, pero su determinación y convicción generó un cambio significativo en la elección de los argentinos, obteniendo el apoyo de la mayoría.  Mi involucramiento como fiscal electoral me brindó esperanza en un futuro próspero para Argentina, aprendiendo mucho de usted y defendiendo sus votos en contra del fraude. Su ascenso representa sin dudas un cambio de paradigma. Desde Jujuy, veo oportunidades para un mejor futuro económico con líderes liberales. Espero que bajo su liderazgo, mi provincia reciba acompañamiento e impulso para su crecimiento. Su triunfo, despierta en mí emociones de esperanza y compromiso con un futuro prometedor. Confío en que liderará el cambio hacia la libertad y el progreso. Mateo Beconi (X: @MateoBeconi) – 19 años – Provincia de La Pampa Estimado Sr Javier Presidente de la República Argentina: En este año nada me hizo más feliz que tu victoria en las últimas elecciones. Fue una sensación increíble de alivio haber presenciado este triunfo, y por eso es de mi agrado enviarle mis felicitaciones a través de esta carta. Quería comentarle que espero con muchas ansias su asunción para con sus nuevas políticas levantar proyectos propios, pudiendo de esta manera fortalecer mi bolsillo. Además, deseo que genere un aura próspera donde se generen las condiciones óptimas para aportar al país mi conocimiento en programación de software. Que Dios y la Patria lo bendigan Javier, pero, por sobre todo, Viva La Libertad Carajo. Firma: Mateo Beconi.  Máximo Morassutti (IG: @politicomorassutti) – 14 años – Provincia de Mendoza “Recuerdo que los cuerpos de intelectuales hablaban de perderlo todo. Hablaban de “democracia” en un país donde reinan censuras y amenazas, de “soberanía” con una moneda sin valor, de “gratuidad y calidad” con hospitales sin insumos, de “derechos” con escuelas públicas deficientes donde no existe incentivo a la mejora y predominan docentes que lejos de educar, adoctrinan. De “dignidad” con jubilados pobres, y de “patria”, en un contexto donde jóvenes, a pesar de poseer todas las capacidades, se ven privados del mundo laboral, limitando el sueño de la casa propia a una notebook. Sin olvidar aquellos que trágicamente ni siquiera llegaron a vislumbrar ese sueño, porque su vida les fue arrebatada en un pestañeo por un motochorro. Hoy en día los argentinos necesitan un fuerte trabajo en dos ámbitos de urgencia: una justicia que verdaderamente se cumpla con la premisa clara: “quien las hace, las paga”, y una economía que permita prever un proyecto de vida: donde los políticos no puedan tapar sus desastres recurriendo a la emisión monetaria, donde podamos estar seguros de que ahorramos en una moneda fuerte, donde no se ceda al gasto entendiendo que los recursos son escasos para necesidades infinitas, donde se permita el libre comercio y se dé lugar a la competencia…” Estimado Presidente Electo Javier Milei. En sus manos reposa la esperanza de la población agotada de trabajar sin poder permitirse la planificación, del productor condenado a los impuestos, de las familias víctimas de la delincuencia como las de Cecilia y de Lucio, de la cara golpeada de la militante Maribel, del jubilado Mario… todos ellos, los argentinos de bien, sintieron romper sus cadenas cuando se anunció tu victoria. La tarea por delante es ardua, pero sé que usted cuenta con la convicción necesaria para no desistir en ninguna de sus propuestas. Quienes entendemos que este país debe cambiar, te proporcionaremos sin dudar la paciencia y el acompañamiento necesario para superar los desafíos, y de esta manera, construir una Argentina libre que resplandezca en todos los aspectos, como lo conseguimos con las ideas alberdianas. Así que adelante, que no te ciegue el miedo de nadie, porque el camino hacia el progreso sólo comienza con pasos firmes. Atentamente, Máximo Morassutti. Aprendiz de Economía. Articulista de críticas y reflexiones políticas.  Nicolás Schell  (IG: schell.nicolas) – 23 años – Provincia de Corrientes Sr. Presidente de la Nación, Javier Gerardo Milei.  Estimado, un placer saludarlo, Soy Nicolás Antonio Schell, presidente de la agrupación “Activistas por la Libertad”, le envío esta carta de agradecimiento por tomarse el tiempo de leernos, así mismo lo felicito por su gloriosa victoria en esta última elección. Usted unió a los argentinos a tal punto que cada uno aportamos en esta campaña un granito de arena porque creemos firmemente en las propuestas liberales. Nosotros los jóvenes esperamos que su gobierno sea exitoso, aunque entendemos las dificultades actuales. Pero estamos aquí para defender las ideas de la libertad como ya lo venimos haciendo desde el 2019, cuando fundamos después de tanto tiempo en la Facultad de Derecho y Ciencias Políticas y Sociales (UNNE) la “Agrupación Liberal Alberdi”. Cómo Correntino, tierra del General San Martín, le pedimos que tenga presente junto con el Gobierno Provincial la realización de acuerdos para que la industria privada creadora del empleo genuino siga en crecimiento, para que el campo sea más libre con menos trabas impositivas, para poder aprovechar al máximo la industria turística en el Iberá y toda la localidad, ya que en región no conocemos el futuro ni progreso porque por malos gobiernos nuestro distrito quedó estancado. Los jóvenes así, en su más mínima posibilidad, abandonan su tierra. Debemos trabajar arduamente para que la libertad llegue por estos lares, para así reconstruir a la región del NEA que tan abandonada se encuentra. Nuestra provincia, siempre discriminada por gobiernos nacionales, hace que le pidamos con fuerza que no se olvide de nosotros, porque cuando la patria está en peligro no se preocupe Sr presidente, que Corrientes le va ayudar. Me despido humildemente de usted. Atentamente, Nicolás.  Pablo Cabrera (IG: @cabrerapabloalexis) – 24 años – Provincia de Entre Ríos Estimado Presidente Electo Javier Milei y Vicepresidente Electa Victoria Villarruel. En nombre de la organización Libertad Avanza Entre Ríos, quiero expresarles nuestro más profundo agradecimiento y admiración por representarnos y liderar un cambio verdadero en nuestro querido país. Su valiente y auténtica campaña política ha despertado un sentimiento de esperanza y renovación en cada rincón de Argentina. Es imposible encontrar palabras suficientes para transmitir la emoción que sentimos al ver plasmado en las urnas el resultado de su incansable trabajo. Lucharon contra vientos adversos, difamaciones y mentiras propias de la vieja política, pero siempre mantuvieron su enfoque en la verdad y en el bienestar de todos los argentinos. Recuerdo con claridad los momentos en los que, como militantes de la libertad, recorríamos las calles más concurridas de la ciudad, repartiendo nuestras boletas, y enfrentando tanto la indiferencia de algunos como el rechazo agresivo de otros. Esos momentos de injusticia y confrontación solo fortalecieron nuestra convicción de que juntos podríamos construir una nación en la que todos pudiéramos sentirnos orgullosos: “Volver a ser potencia mundial” era el grito que entonábamos en cada esquina, convencidos de que era posible alcanzarlo. Es notorio y encomiable el genuino esfuerzo y compromiso que tanto usted, Presidente Electo Milei, como usted, Vicepresidenta Electa Villarruel, han demostrado a lo largo de esta campaña. Aunque carecían de experiencia política, sobraba en ustedes la valentía, la entrega y las ganas de lograr un cambio profundo en nuestra querida nación. Agradecemos especialmente a todas las personas que confiaron en nuestras propuestas y no se dejaron llevar por la campaña del miedo. Fueron ustedes quienes votaron por una Argentina mejor, una Argentina en la que todos tengamos las mismas oportunidades y en la que la libertad sea el motor de nuestro progreso. Nuestro agradecimiento también se extiende a todos los militantes de la libertad que trabajaron arduamente, día y noche, para hacer posible este resultado histórico. Hoy, con la victoria en nuestras manos, nos enfrentamos a una nueva y emocionante etapa: la reconstrucción de nuestro país. Sabemos que el camino no será fácil, pero estamos seguros de que, con su liderazgo y el compromiso de todos los argentinos, lograremos recuperar el esplendor de nuestro suelo, el cual supo ser el más próspero del nuevo mundo. En cada uno de nosotros está la responsabilidad de trabajar por un futuro brillante y justo para todos. Continuaremos junto a ustedes, apoyando las medidas y políticas que promuevan la libertad, el desarrollo económico y la igualdad de oportunidades. Una vez más, les expresamos nuestro más sincero agradecimiento por representarnos y ser la voz de aquellos que anhelan un cambio real en nuestra Argentina. Estamos convencidos de que su liderazgo marcará una diferencia trascendental en nuestro país. Con el más profundo respeto y admiración, Atte: Cabrera Pablo, Libertad Avanza Entre Ríos.  Patricio Aguilera Roque (X: @Mr_Desfase) – 19 años – Provincia de Catamarca Querido Javier Milei, Permítame expresar mi más sincero y emocionado agradecimiento por su reciente elección como presidente de Argentina. Tu victoria ha traído consigo un torrente de esperanza y entusiasmo para muchos de nosotros que anhelamos un cambio significativo en Argentina. Tu intelecto, valentía y pasión inspiran a muchos, incluyéndome. Es con gran entusiasmo que te brindo mi apoyo y confianza en tu capacidad para dirigir nuestro país hacia un futuro próspero y recuperar la potencia que una vez fuimos. Con gratitud y esperanza, Roque Patricio Aguilera Santiago Vieites (IG: @santiago_vieites2605) – 18 años – Provincia de Buenos Aires “Milei representa el hartazgo y a su vez la esperanza de una sociedad atrofiada por el populismo político. Lo veo a Javier como el inicio de un camino, el líder de una verdadera revolución. Nuestro país ha sido víctima de muchos gobiernos los cuales solo han buscado el poder con el fin de enriquecerse a sí mismos, han dejado gente en la pobreza, familias sin hogar, y utilizaron esas figuras como símbolos políticos promoviendo la grieta entre todos nosotros.  Lo veo como la persona que demostró que la grieta no es entre ciudadanos, sino entre los políticos que han malversado la administración pública y los ciudadanos de este país que han trabajado insaciablemente solo para pagarle todos los lujos al Estado.  Me gustaría ver, en estos 4 años por lo menos, el inicio de un cambio, las primeras luces verdes de nuestro camino. Quisiera que muchos de los delincuentes, a los que Javier correctamente llama casta, paguen por las atrocidades que le hicieron a la patria. Quisiera ver un país donde los jóvenes empiecen a replantearse la idea de irse del país, y decidan, justificadamente, quedarse. Quisiera ver un país donde se cumpla y se respete la ley. Quisiera ver un país más floreciente, donde no se trabaje como esclavo para entregarle el fruto de tu trabajo al gobierno, sino, uno donde los argentinos gocen de libertad y prosperidad en sus vidas. Estas elecciones fueron claras, a pesar de que el resultado de las elecciones generales fue sorprendente, el balotaje lo dejó todo bastante claro. En ningún momento de la campaña dejé de respirar aires de cambio y libertad. Los argentinos eligieron a la persona que creen más capaz de liderar este cambio, tanto fue así que Javier Milei se convirtió en el presidente más votado en la historia de nuestra democracia moderna.  Santiago Benítez (X: @Santitoo_03) – 20 años – Provincia de Formosa Estimado señor presidente Me dirijo a usted en calidad de ciudadano residente de la provincia de Formosa con el agrado de primero felicitarlo por ser el nuevo presidente electo de la República Argentina, cuenta con todo mi apoyo y el de la mayoría del pueblo argentino, juntos le ayudaremos a que este país ocupe el lugar en el mundo que nunca debió dejar, defendiendo la competencia, la libertad y la propiedad privada.  Muchas gracias por su atención. Atentamente, Benitez Santiago Emanuel Epílogo – Por Jeremías Rucci (X: @jeremias_rucci) – 21 años Queridos Máximo Morasutti y Lionel Schroeder; Estimado Javier, Es un honor para mí expresar mi agradecimiento por la amable invitación a poner el epílogo a esta carta en respaldo al presidente electo liberal libertario, Javier Milei. Los más de 23 jóvenes que colaboramos en esta carta lo hicimos impulsados por la esperanza de ver la libertad como el catalizador del cambio para nuestra amada nación. Javier Milei encarna la esperanza de millones de argentinos que lograron derribar el muro del kirchnerismo, que durante más de 20 años despojó a nuestra nación de sueños y esperanzas. Por eso, apreciado Javier, a partir del 10 de diciembre de 2023, con tu liderazgo y el respaldo de los argentinos, comienza un nuevo capítulo para Argentina. Una etapa llena de vigor, repleta de esperanza y libertad. Este nuevo amanecer es un llamado a la acción, una invitación a explorar horizontes de progreso y bienestar a través de los principios fundamentales del liberalismo. La travesía que emprenderás como líder de nuestra nación no estará exenta de desafíos. Enfrentarás críticas, resistencia y obstáculos, pero confiamos en que tu determinación y visión llevarán a Argentina hacia un futuro más brillante. La responsabilidad que asumes es monumental, pero también es una oportunidad única para redefinir el rumbo de nuestro país, devolviendo el poder a los ciudadanos y construyendo un legado de libertad y prosperidad para las generaciones venideras. Este momento crucial exige que enfrentemos los desafíos con valentía y determinación. La historia nos ha enseñado que los mejores momentos de nuestra amada Nación fueron con Libertad, donde las botas del estado estaban lejos de los ciudadanos. En esta nueva etapa, te instamos a mantener la llama de la libertad ardiendo, a defenderla contra vientos contrarios y a trabajar incansablemente por un país donde cada argentino pueda perseguir sus sueños sin miedo a la opresión. Que la libertad sea el hilo conductor que te guíe en la toma de decisiones, recordándonos a todos que la grandeza de una nación radica en la libertad de sus ciudadanos. La oportunidad de forjar un futuro mejor está frente a nosotros, Javier. El momento es ahora o nunca.  Lapidario informe del Massazo: Los ultimos numeros muestran que Massa dejó un salario mínimo de 152 dólares en Argentina Gracias al DNU de Milei, las empresas están dejando de retener obligatoriamente la cuota sindical del sueldo de sus empleados Empezó la Motosierra en la TV Pública: Se eliminaron dos noticieros, móviles a la costa y toda la grilla militante Con solo 20 días en el cargo y la Ciudad en emergencia, Jorge Macri se fue de vacaciones a Punta del Este en jet privado Estallido social en La Rioja de Quintela: La policía y los empleados públicos denuncian salarios impagos Gracias a las medidas de Milei, se abren nuevos mercados en Brasil y China para la exportación de productos argentinos Periodista que expuso el Pizzagate y produjo un documental en contra de una red global de pedófilos, fue hallada muerta Cavallo acertó la inflación de diciembre con una increíble exactitud, y anticipa que volverá a un dígito mensual para el mes de abril ¡CONTAMOS CON TU AYUDA! Este proyecto, este equipo, y este diario son sólo posibles gracias a la contribución de cientos de donaciones de nuestros lectores todos los días. CONTACTO: Para contactarse con nosotros: contacto@maderomediagroup.com  Copyright © 2023."
result = spanish_lemmatizer_with_pos(text_to_lemmatize)

# Print the lemmatized text
print(result)


Argentina y uno giro transcendental en su historia : el voto a Milei ser el búsqueda de el cura para el enfermedad , https://derechadiario.com.ar / opinion / argentina-y-un-giro-transcendental-en-su-historia-el-voto-a-milei-es-la-busqueda-de-la-cura-para-la-enfermedad , laderechadiario , right_wing , Publicado en Por el diputado nacional ser el candidato más votado en el primaria y agradecer “ a todo el que venir apostar desde el 2021 en crear uno proyecto liberal , con proyección nacional y que llegar a ser gobierno ” .    el comicio del pasado domingo 13 de agosto en Argentina no pasar desapercibido en su historia . suceder que el candidato libertario Javier Milei dar el batacazo imponer él en el urna por encima de el dos fuerza política hegemónico , Juntos por el Cambio , respaldado por el UCR , y Unión por el Patria , respaldado por el PJ .   si bien ser cierto que este no ser algo definitivo dado que él tratar de elección primario y el distancia entre el tres primero candidato no 